# Install necessary Libraries

In [ ]:
!pip install -U langchain-community
!pip install sentence_transformers
!pip install pinecone-client
!pip install pinecone-client[grpc]
!pip install nltk
!pip install beautifulsoup4

In [ ]:
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

api_key = "6ca3b069-c0d4-460e-8d64-9f545902b7c6"  
environment = "us-east-1"  

pc = Pinecone(api_key=api_key)

index_name = "chunk-embeddings-index"
dimension = 768  

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region=environment
        ) 
    )

index = pc.Index(index_name)

# Load Data, Chunk Legal Documents for Processing and setting up the vector database


In [ ]:
import numpy as np
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz")
embedding_model = AutoModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
text_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

chunks = []

def chunk_text_based_on_tokens(text, max_tokens=300):
    sentences = sent_tokenize(text)
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(tokenizer.tokenize(sentence))
        if current_length + sentence_length <= max_tokens:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

def extract_sections_articles_chapters(soup):
    sections = []
    current_section = []
    for element in soup.find_all(['h1', 'h2', 'h3', 'p']):
        if element.name in ['h1', 'h2', 'h3']:
            if current_section:
                sections.append(" ".join(current_section))
                current_section = []
            current_section.append(element.get_text())
        else:
            current_section.append(element.get_text())
    if current_section:
        sections.append(" ".join(current_section))
    return sections

def load_and_process_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = extract_sections_articles_chapters(soup)
    all_chunks = []
    for section in sections:
        all_chunks.extend(chunk_text_based_on_tokens(section))
    return all_chunks

def generate_bert_embedding(text, tokenizer, model, max_length=512):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True, 
        padding=True,
        max_length=max_length  
    )
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).numpy()
    return embedding.tolist()

def upsert_chunks_to_pinecone(index, namespace, chunks, batch_size=10):
    vectors = []
    print(here)
    for i, chunk in enumerate(chunks):
        embedding = generate_bert_embedding(chunk, text_tokenizer, embedding_model)
        vectors.append({
            "id": f"{namespace}_id_{i}", 
            "values": embedding,
            "metadata": {"text": chunk}  
        })

        if len(vectors) >= batch_size:
            index.upsert(vectors=vectors, namespace=namespace)
            vectors = []  

    if vectors:
        index.upsert(vectors=vectors, namespace=namespace)
        print(f"Upserted {len(vectors)} remaining vectors to namespace '{namespace}'.")

def process_and_upload_html(file_path, namespace):
    print(f"Processing file: {file_path} for namespace: {namespace}")
    chunks = load_and_process_html(file_path) 
    upsert_chunks_to_pinecone(index, namespace, chunks) 

html_files_and_namespaces = {
    'gdpr': '/kaggle/input/greek-laws-and-qa/greek_gdpr.html',
    'ai_act': '/kaggle/input/greek-laws-and-qa/greek_AI_act.html',
    'dma': '/kaggle/input/greek-laws-and-qa/greek_dma.html',
    'dsa': '/kaggle/input/greek-laws-and-qa/greek_dsa.html'
}

for namespace, file_path in html_files_and_namespaces.items():
    process_and_upload_html(file_path, namespace)

In [ ]:
from sentence_transformers import SentenceTransformer

def embed_chunks(chunks, model_name):
    model = SentenceTransformer(model_name, device='cpu')
    embeddings = model.encode(chunks, normalize_embeddings=True)
    return embeddings

def embed_query(query, model_name):
    model = SentenceTransformer(model_name, device='cpu')
    query_embedding = model.encode([query], normalize_embeddings=True)
    return query_embedding[0]



def process_and_store_embeddings(file_path, namespace, model_name):
    chunks = load_and_process_html(file_path)
    embeddings = embed_chunks(chunks, model_name)

    vectors_to_upsert = [(f"id_{namespace}_{i}", embedding.tolist()) for i, embedding in enumerate(embeddings)]
    index.upsert(vectors=vectors_to_upsert, namespace=law)  

    return index, chunks, embeddings


def query_pinecone_db(query_embedding, index, namespace, top_k=1):
    results = index.query(vector=query_embedding, top_k=top_k, namespace=law)  
    return results

# retrieve the most relevant chunk based on testing specific queries

In [ ]:
laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_gdpr.html',
        'index_name': 'index_name',
        'query': "Πώς συμβάλλει ο παρών κανονισμός στη διασφάλισ/kaggle/input/greek-laws-and-qa/greek_gdpr.htmlη της ελεύθερης κυκλοφορίας των δεδομένων προσωπικού χαρακτήρα στην εσωτερική αγορά της Ευρωπαϊκής Ένωσης, και ποιες προβλέψεις περιλαμβάνονται για τις πολύ μικρές, μικρές και μεσαίες επιχειρήσεις;"
    },
    'ai_act': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_AI_act.html',
        'index_name': 'index_name',
        'query': "Πώς ο παρών κανονισμός διασφαλίζει ότι οι εναρμονισμένοι κανόνες για τα συστήματα Τεχνητής Νοημοσύνης υψηλού κινδύνου δεν θα θίγουν τα υφιστάμενα δικαιώματα των εργαζομένων και την εφαρμογή του εθνικού εργατικού δικαίου;"
    },
    'dma': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_dma.html',
        'index_name': 'index_name',
        'query': "Ποια είναι η διαδικασία που προβλέπει ο κανονισμός για την ενημέρωση των παραρτημάτων Ι, ΙΙ και IV και ποια είναι η σημασία των διαβουλεύσεων που διεξάγει η Επιτροπή κατά τις προπαρασκευαστικές εργασίες για την έκδοση των κατ’ εξουσιοδότηση πράξεων;"
    },
    'dsa': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_dsa.html',
        'index_name': 'index_name',
        'query': "Πώς ο παρών κανονισμός αλληλεπιδρά με άλλες ενωσιακές νομικές πράξεις που ρυθμίζουν την παροχή υπηρεσιών της κοινωνίας της πληροφορίας και την προστασία των καταναλωτών, και σε ποιες περιπτώσεις υπερισχύει ο παρών κανονισμός όταν οι στόχοι των άλλων νομικών πράξεων συμπίπτουν με τους στόχους του;"
    }
}

model_name = "nlpaueb/bert-base-greek-uncased-v1"

chunks_dict = {}
embeddings_dict = {}
import time

if index_name in pc.list_indexes().names():
    print(f"Deleting index '{index_name}'...")
    pc.delete_index(index_name)

    print("Waiting for the index to be deleted...")
    time.sleep(5)  

    if index_name not in pc.list_indexes().names():
        print(f"Index '{index_name}' successfully deleted.")
    else:
        print(f"Index '{index_name}' still exists, trying again...")
        pc.delete_index(index_name)
        time.sleep(5) 
        if index_name not in pc.list_indexes().names():
            print(f"Index '{index_name}' successfully deleted on second attempt.")
        else:
            print(f"Failed to delete index '{index_name}'. Please check your Pinecone dashboard.")
else:
    print(f"Index '{index_name}' does not exist or was already deleted.")

pc.create_index(
    name=index_name,
    dimension=dimension,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region=environment
    )
)
print(f"Index '{index_name}' created successfully.")

for law, info in laws_info.items():
    print(f"Processing {law}...")
    index, chunks, embeddings = process_and_store_embeddings(info['file_path'], info['index_name'], model_name)
    chunks_dict[law] = chunks  
    embeddings_dict[law] = embeddings

    vectors_to_upsert = [(f"{law}_id_{i}", embedding.tolist()) for i, embedding in enumerate(embeddings)]
    
    try:
        print(f"Clearing vectors for namespace '{law}'...")
        index.delete(delete_all=True, namespace=law)  
        print(f"Namespace '{law}' cleared.")
    except Exception as e:
        print(f"Namespace '{law}' does not exist or an error occurred: {e}")
    
    try:
        print(f"Upserting vectors for namespace '{law}'...")
        index.upsert(vectors=vectors_to_upsert, namespace=law)
        print(f"Upsert complete for namespace '{law}'.")
    except Exception as e:
        print(f"Failed to upsert vectors for namespace '{law}': {e}")

print("Chunks Dict Content: ")
for key in chunks_dict.keys():
    print(f"Law: {key}, Number of Chunks: {len(chunks_dict[key])}")

In [ ]:
for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} index:")
    query_embedding = embed_query(info['query'], model_name)
    print(info['query'])
    results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1) 
    
    if results and 'matches' in results and results['matches']:
        retrieved_chunk_id = results['matches'][0]['id']
        retrieved_chunk = chunks[law][int(retrieved_chunk_id.split('_')[-1])]
        print(f"Retrieved chunk {retrieved_chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_chunk)
    else:
        print(f"No results found for {law.upper()}.")

# measuring the relevancy of the retrieved texts and the answers

In [ ]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, util

tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")
model = AutoModel.from_pretrained("nlpaueb/bert-base-greek-uncased-v1")

semantic_model = SentenceTransformer('nlpaueb/bert-base-greek-uncased-v1', device='cuda')

def generate_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].numpy() 
    return embedding

def calculate_cosine_similarity(reference_embedding, retrieved_embedding):
    return cosine_similarity(reference_embedding.reshape(1, -1), retrieved_embedding.reshape(1, -1))[0][0]

def calculate_semantic_similarity(reference_text, retrieved_text, model):
    embeddings1 = model.encode(reference_text, convert_to_tensor=True)
    embeddings2 = model.encode(retrieved_text, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
    return similarity.item()

reference_answers = {
    'gdpr': " Για να διασφαλιστεί συνεκτικό επίπεδο προστασίας των φυσικών προσώπων σε ολόκληρη την Ένωση και προς αποφυγή αποκλίσεων που εμποδίζουν την ελεύθερη κυκλοφορία των δεδομένων προσωπικού χαρακτήρα στην εσωτερική αγορά, απαιτείται κανονισμός ο οποίος θα κατοχυρώνει την ασφάλεια δικαίου και τη διαφάνεια για τους οικονομικούς παράγοντες, περιλαμβανομένων των πολύ μικρών, των μικρών και των μεσαίων επιχειρήσεων, και θα προβλέπει για τα φυσικά πρόσωπα σε όλα τα κράτη μέλη το ίδιο επίπεδο νομικά εκτελεστών δικαιωμάτων και υποχρεώσεων, καθώς και ευθυνών για τους υπευθύνους επεξεργασίας και τους εκτελούντες την επεξεργασία, ώστε να διασφαλιστεί η συνεκτική παρακολούθηση της επεξεργασίας δεδομένων προσωπικού χαρακτήρα, καθώς και οι ισοδύναμες κυρώσεις σε όλα τα κράτη μέλη και η αποτελεσματική συνεργασία μεταξύ των εποπτικών αρχών των διάφορων κρατών μελών. Για την ομαλή λειτουργία της εσωτερικής αγοράς, η ελεύθερη κυκλοφορία δεδομένων προσωπικού χαρακτήρα εντός της Ένωσης δεν πρέπει να περιορίζεται, ούτε να απαγορεύεται για λόγους που σχετίζονται με την προστασία των φυσικών προσώπων έναντι της επεξεργασίας δεδομένων προσωπικού χαρακτήρα. Για να ληφθεί υπόψη η ειδική κατάσταση των πολύ μικρών, των μικρών και των μεσαίων επιχειρήσεων, ο παρών κανονισμός περιλαμβάνει παρέκκλιση για οργανισμούς που απασχολούν λιγότερα από 250 άτομα όσον αφορά την τήρηση αρχείων. Επιπλέον, τα θεσμικά όργανα και οι οργανισμοί της Ένωσης, καθώς και τα κράτη μέλη και οι εποπτικές αρχές τους, παροτρύνονται να λαμβάνουν υπόψη τις ειδικές ανάγκες των πολύ μικρών, των μικρών και των μεσαίων επιχειρήσεων κατά την εφαρμογή του παρόντος κανονισμού. Η έννοια των πολύ μικρών, των μικρών και των μεσαίων επιχειρήσεων θα πρέπει να βασίζεται στο άρθρο 2 του παραρτήματος στη σύσταση 2003/361/ΕΚ της Επιτροπής (5).",
    'ai_act': "Η έννοια του «συστήματος ΤΝ» στον παρόντα κανονισμό θα πρέπει να ορίζεται με σαφήνεια και να είναι στενά ευθυγραμμισμένη με το έργο διεθνών οργανισμών που ασχολούνται με την TN, με στόχο την ασφάλεια δικαίου και τη διευκόλυνση της διεθνούς σύγκλισης και της ευρείας αποδοχής, παρέχοντας παράλληλα ευελιξία για την προσαρμογή στις ταχείες τεχνολογικές εξελίξεις στον τομέα αυτόν. Επιπλέον, ο ορισμός θα πρέπει να στηρίζεται σε βασικά χαρακτηριστικά των συστημάτων TN τα οποία διακρίνουν τα συστήματα αυτά από απλούστερα παραδοσιακά συστήματα λογισμικού ή προσεγγίσεις προγραμματισμού, δεν θα πρέπει δε να καλύπτει συστήματα που βασίζονται στους κανόνες που ορίζονται αποκλειστικά από φυσικά πρόσωπα με σκοπό την αυτόματη εκτέλεση λειτουργιών. Βασικό χαρακτηριστικό των συστημάτων ΤΝ είναι η ικανότητά τους να συνάγουν συμπεράσματα. Η ικανότητα συναγωγής συμπερασμάτων αναφέρεται στη διαδικασία απόκτησης των στοιχείων εξόδου, όπως προβλέψεων, περιεχομένου, συστάσεων ή αποφάσεων, που μπορούν να επηρεάσουν υλικά και εικονικά περιβάλλοντα, καθώς και στην ικανότητα των συστημάτων ΤΝ να εξάγουν μοντέλα ή αλγορίθμους ή και τα δύο από στοιχεία εισόδου ή δεδομένα. Στις τεχνικές που επιτρέπουν τη συναγωγή συμπερασμάτων κατά τη δημιουργία ενός συστήματος ΤΝ περιλαμβάνονται προσεγγίσεις μηχανικής μάθησης που μαθαίνουν από δεδομένα πώς να επιτυγχάνουν ορισμένους στόχους, καθώς και προσεγγίσεις βασιζόμενες στη λογική και τη γνώση οι οποίες συνάγουν συμπεράσματα από κωδικοποιημένη γνώση ή συμβολική αναπαράσταση της προς εκτέλεση εργασίας. Η ικανότητα ενός συστήματος ΤΝ να συνάγει συμπεράσματα υπερβαίνει τη βασική επεξεργασία δεδομένων επιτρέποντας τη μάθηση, τη συλλογιστική ή τη μοντελοποίηση. Ο όρος «μηχανικά» αναφέρεται στο γεγονός ότι τα συστήματα ΤΝ λειτουργούν σε μηχανήματα. Η αναφορά σε ρητούς ή σιωπηρούς στόχους υπογραμμίζει ότι τα συστήματα ΤΝ μπορούν να λειτουργούν σύμφωνα με ρητούς καθορισμένους στόχους ή με σιωπηρούς στόχους. Οι στόχοι του συστήματος ΤΝ μπορεί να διαφέρουν από τον επιδιωκόμενο σκοπό του συστήματος ΤΝ σε ένα συγκεκριμένο πλαίσιο. Για τους σκοπούς του παρόντος κανονισμού, ως «περιβάλλοντα» θα πρέπει να νοούνται τα πλαίσια εντός των οποίων λειτουργούν τα συστήματα ΤΝ, τα δε στοιχεία εξόδου που παράγονται από το σύστημα ΤΝ αντικατοπτρίζουν διαφορετικές λειτουργίες επιτελούμενες από τα συστήματα ΤΝ και περιλαμβάνουν προβλέψεις, περιεχόμενο, συστάσεις ή αποφάσεις. Τα συστήματα ΤΝ σχεδιάζονται ώστε να λειτουργούν με ποικίλα επίπεδα αυτονομίας, πράγμα που σημαίνει ότι διαθέτουν ορισμένο βαθμό ανεξαρτησίας να δρουν χωρίς ανθρώπινη συμμετοχή και ορισμένο βαθμό ικανότητας να λειτουργούν χωρίς ανθρώπινη παρέμβαση. Η προσαρμοστικότητα που θα μπορούσε να επιδείξει ένα σύστημα ΤΝ μετά την εφαρμογή του αναφέρεται σε ικανότητες αυτομάθησης οι οποίες επιτρέπουν στο σύστημα να αλλάζει ενώ χρησιμοποιείται. Τα συστήματα ΤΝ μπορούν να χρησιμοποιούνται σε αυτόνομη βάση ή ως κατασκευαστικό στοιχείο προϊόντος, ανεξάρτητα από το αν το σύστημα έχει ενσωματωθεί υλικά στο προϊόν (ενσωματωμένο σύστημα) ή εξυπηρετεί τη λειτουργικότητα του προϊόντος χωρίς να έχει ενσωματωθεί σε αυτό (μη ενσωματωμένο σύστημα).",
    'dma': " Για να καταστεί δυνατή η ταχεία αντίδραση της Ένωσης στις μεταβαλλόμενες περιστάσεις σε ό,τι αφορά την αξιολόγηση της ευαισθησίας των εξαγωγών που πραγματοποιούνται με βάση ενωσιακές γενικές άδειες εξαγωγής, αλλά και στις τεχνολογικές και εμπορικές εξελίξεις, σύμφωνα με το άρθρο 290 της Συνθήκης για τη λειτουργία της Ευρωπαϊκής Ένωσης (ΣΛΕΕ) θα πρέπει να ανατεθεί στην Επιτροπή η αρμοδιότητα να εκδίδει πράξεις τροποποίησης των παραρτημάτων Ι, ΙΙ και IV του παρόντος κανονισμού. Οι αποφάσεις για την ενημέρωση του κοινού καταλόγου ειδών διπλής χρήσης που υπόκεινται σε ελέγχους εξαγωγών που παρατίθεται στο παράρτημα I θα πρέπει να είναι σύμφωνες με τις υποχρεώσεις και δεσμεύσεις που έχουν αναλάβει τα κράτη μέλη ή η Ένωση ως συμβαλλόμενα μέρη των οικείων διεθνών συμφωνιών μη διάδοσης και ως συμβαλλόμενα μέρη των πολυμερών συστημάτων ελέγχου των εξαγωγών ή με την κύρωση των οικείων διεθνών συνθηκών. Όταν η τροποποίηση του παραρτήματος Ι αφορά ειδη διπλής χρήσης που απαριθμούνται επίσης στο παράρτημα ΙΙ ή IV, τα εν λόγω παραρτήματα θα πρέπει να τροποποιηθούν αναλόγως. Οι αποφάσεις για την ενημέρωση των κοινών καταλόγων ειδών και προορισμών που παρατίθενται στα τμήματα Α έως Η του παραρτήματος ΙΙ θα πρέπει να λαμβάνονται με συνεκτίμηση των κριτηρίων αξιολόγησης που περιγράφονται στον παρόντα κανονισμό. Είναι ιδιαίτερα σημαντικό η Επιτροπή να διεξάγει, κατά τις προπαρασκευαστικές της εργασίες, τις κατάλληλες διαβουλεύσεις, μεταξύ άλλων σε επίπεδο εμπειρογνωμόνων, και οι εν λόγω διαβουλεύσεις να διενεργούνται σύμφωνα με τις αρχές που ορίζονται στη διοργανική συμφωνία της 13ης Απριλίου 2016 για τη βελτίωση του νομοθετικού έργου (5). Πιο συγκεκριμένα, προκειμένου να εξασφαλιστεί η ισότιμη συμμετοχή στην προετοιμασία των κατ’ εξουσιοδότηση πράξεων, το Ευρωπαϊκό Κοινοβούλιο και το Συμβούλιο λαμβάνουν όλα τα έγγραφα κατά τον ίδιο χρόνο με τους εμπειρογνώμονες των κρατών μελών, και οι εμπειρογνώμονές τους έχουν συστηματικά πρόσβαση στις συνεδριάσεις των ομάδων εμπειρογνωμόνων της Επιτροπής που ασχολούνται με την προετοιμασία κατ’ εξουσιοδότηση πράξεων.",
    'dsa': " Προκειμένου να επιτευχθεί ο στόχος της διασφάλισης ενός ασφαλούς, προβλέψιμου και αξιόπιστου επιγραμμικού περιβάλλοντος, για τους σκοπούς του παρόντος κανονισμού η έννοια του «παράνομου περιεχομένου» θα πρέπει να αντικατοπτρίζει σε γενικές γραμμές τους κανόνες που υφίστανται στο εκτός διαδικτύου περιβάλλον, Ειδικότερα, η έννοια του «παράνομου περιεχομένου» θα πρέπει να οριστεί ευρέως ώστε να καλύπτει πληροφορίες που αφορούν παράνομο περιεχόμενο, προϊόντα, υπηρεσίες και δραστηριότητες. Ειδικότερα, η έννοια αυτή θα πρέπει να νοείται ως αναφορά σε πληροφορίες, ανεξάρτητα από τη μορφή τους, οι οποίες, σύμφωνα με το εφαρμοστέο δίκαιο, είτε είναι οι ίδιες παράνομες, όπως παράνομη ρητορική μίσους ή τρομοκρατικό περιεχόμενο και παράνομο περιεχόμενο που εισάγει διακρίσεις, είτε οι εφαρμοστέοι κανόνες τις καθιστούν παράνομες λόγω του ότι αφορούν παράνομες δραστηριότητες. Ενδεικτικά παραδείγματα αποτελούν η κοινοποίηση εικόνων που απεικονίζουν σεξουαλική κακοποίηση παιδιών, η παράνομη μη συναινετική κοινοποίηση ιδιωτικών εικόνων, η παρενοχλητική παρακολούθηση στο διαδίκτυο, η πώληση μη συμμορφούμενων προϊόντων ή προϊόντων παραποίησης/απομίμησης, η πώληση προϊόντων ή η παροχή υπηρεσιών κατά παράβαση της νομοθεσίας για την προστασία των καταναλωτών, η μη εξουσιοδοτημένη χρήση υλικού που προστατεύεται από δικαιώματα πνευματικής ιδιοκτησίας, η παράνομη προσφορά υπηρεσιών στέγασης ή η παράνομη πώληση ζώντων ζώων. Αντιθέτως, ένα βίντεο αυτόπτη μάρτυρα σε ένα πιθανό έγκλημα δεν θα πρέπει να θεωρείται ότι αποτελεί παράνομο περιεχόμενο απλώς και μόνο επειδή απεικονίζει παράνομη πράξη, όταν η εγγραφή ή η διάδοση τέτοιου βίντεο στο κοινό δεν είναι παράνομη βάσει του εθνικού ή του ενωσιακού δικαίου. Στο πλαίσιο αυτό, δεν έχει σημασία αν ο παράνομος χαρακτήρας των πληροφοριών ή της δραστηριότητας προκύπτει από το ενωσιακό δίκαιο ή από εθνικό δίκαιο το οποίο συμμορφώνεται προς το ενωσιακό δίκαιο, ούτε έχει σημασία η ακριβής φύση ή το αντικείμενο του επίμαχου δικαίου."
}

In [ ]:
similarities = []

for law, info in laws_info.items():
    print(f"\nQuerying {law.upper()} index:")
    query_embedding = embed_query(info['query'], model_name)
    print(info['query'])
    results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1) 
    
    if results and 'matches' in results and results['matches']:
        retrieved_chunk_id = results['matches'][0]['id']
        retrieved_chunk = chunks_dict[law][int(retrieved_chunk_id.split('_')[-1])]
        print(f"Retrieved chunk {retrieved_chunk_id.split('_')[-1]} from {law.upper()}:")
        print(retrieved_chunk)

        retrieved_embedding = generate_bert_embedding(retrieved_chunk, tokenizer, model)
        reference_embedding = generate_bert_embedding(reference_answers[law], tokenizer, model)
        cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)
        
        semantic_sim = calculate_semantic_similarity(reference_answers[law], retrieved_chunk, semantic_model)

        similarities.append({
            'law': law,
            'retrieved_answer': retrieved_chunk,
            'cosine_similarity': cosine_sim,
            'semantic_similarity': semantic_sim
        })

        print(f"Cosine Similarity with reference answer: {cosine_sim:.4f}")
        print(f"Semantic Similarity with reference answer: {semantic_sim:.4f}")
        print("----\n")
    else:
        print(f"No results found for {law.upper()} in the query.")

# setting 20 questions and their answers for each law, make embedidngs of them while making summarizations of the most relevant chunks that have beed retrieved.

In [ ]:
integrated_questions_answers = [
{
    'law': 'gdpr',
    'question': "Ποιο είναι το θεμελιώδες δικαίωμα που αφορά την επεξεργασία δεδομένων προσωπικού χαρακτήρα σύμφωνα με τον Χάρτη Θεμελιωδών Δικαιωμάτων της Ευρωπαϊκής Ένωσης;",
    'answer': "Η προστασία των φυσικών προσώπων σε σχέση με την επεξεργασία δεδομένων προσωπικού χαρακτήρα αποτελεί θεμελιώδες δικαίωμα. Το άρθρο 8(1) του Χάρτη Θεμελιωδών Δικαιωμάτων της Ευρωπαϊκής Ένωσης ('ο Χάρτης') και το άρθρο 16(1) της Συνθήκης για τη Λειτουργία της Ευρωπαϊκής Ένωσης (ΣΛΕΕ) ορίζουν ότι ο καθένας έχει το δικαίωμα προστασίας των προσωπικών του δεδομένων. Ο παρών Κανονισμός αποσκοπεί στη συμβολή στην επίτευξη ενός χώρου ελευθερίας, ασφάλειας και δικαιοσύνης, και μιας οικονομικής ένωσης, στην οικονομική και κοινωνική πρόοδο, στην ενίσχυση και σύγκλιση των οικονομιών εντός της εσωτερικής αγοράς και στην ευημερία των φυσικών προσώπων."
},
{
    'law': 'ai_act',
    'question': "Ποιοι είναι οι κύριοι στόχοι του Νόμου για την Τεχνητή Νοημοσύνη σχετικά με την ανάπτυξη και τη χρήση της τεχνητής νοημοσύνης στην Ευρωπαϊκή Ένωση;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη στοχεύει να διασφαλίσει ότι τα συστήματα τεχνητής νοημοσύνης που τοποθετούνται στην αγορά και χρησιμοποιούνται στην Ένωση είναι ασφαλή, σέβονται την υπάρχουσα νομοθεσία για τα θεμελιώδη δικαιώματα και τις αξίες της Ένωσης και δεν υπονομεύουν τα θεμελιώδη δικαιώματα. Ο νόμος στοχεύει να θεσπίσει ένα νομικό πλαίσιο που αντιμετωπίζει τους κινδύνους που θέτει η τεχνητή νοημοσύνη, ιδίως τα συστήματα υψηλού κινδύνου, και να ενισχύσει τη διαφάνεια, τη λογοδοσία και την εμπιστοσύνη στην τεχνητή νοημοσύνη, προωθώντας παράλληλα την καινοτομία και την ανταγωνιστικότητα."
},
{
    'law': 'dma',
    'question': "Ποια κριτήρια χρησιμοποιούνται για να οριστεί ένας 'διαμεσολαβητής' βάσει του Νόμου για τις Ψηφιακές Αγορές;",
    'answer': "Ένας διαμεσολαβητής βάσει του DMA ορίζεται ως πάροχος βασικών υπηρεσιών πλατφόρμας που έχει σημαντική επίδραση στην εσωτερική αγορά, λειτουργεί ως σημαντική πύλη για τους επιχειρηματικούς χρήστες για να φτάσουν στους τελικούς χρήστες και απολαμβάνει μια παγιωμένη και διαρκή θέση στην αγορά. Τα κριτήρια περιλαμβάνουν την ύπαρξη ισχυρής οικονομικής θέσης, μεγάλο αριθμό χρηστών και τον έλεγχο ενός οικοσυστήματος που είναι δύσκολο να αμφισβητηθεί από άλλες εταιρείες."
},
{
    'law': 'dsa',
    'question': "Ποιες είναι οι κύριες ευθύνες των διαδικτυακών πλατφορμών βάσει του Νόμου για τις Ψηφιακές Υπηρεσίες (DSA);",
    'answer': "Σύμφωνα με τον DSA, οι διαδικτυακές πλατφόρμες είναι υπεύθυνες για τη λήψη αποτελεσματικών μέτρων για τον μετριασμό των κινδύνων που σχετίζονται με παράνομο περιεχόμενο, την εξασφάλιση της ασφάλειας των χρηστών και την προστασία των θεμελιωδών δικαιωμάτων. Οι πλατφόρμες πρέπει να εφαρμόζουν μηχανισμούς αναφοράς και αφαίρεσης παράνομου περιεχομένου, να παρέχουν σαφείς όρους και προϋποθέσεις στους χρήστες και να θεσπίζουν διαδικασίες για τη διαχείριση παραπόνων και ενστάσεων. Οι πλατφόρμες που φτάνουν σε σημαντικό αριθμό χρηστών υποχρεούνται επίσης να αξιολογούν και να μετριάζουν τους συστημικούς κινδύνους, όπως η διάδοση παραπληροφόρησης και επιβλαβούς περιεχομένου."
},
{
    'law': 'gdpr',
    'question': "Πώς σκοπεύει ο GDPR να εξισορροπήσει το δικαίωμα προστασίας των δεδομένων προσωπικού χαρακτήρα με άλλα θεμελιώδη δικαιώματα;",
    'answer': "Ο παρών Κανονισμός σέβεται όλα τα θεμελιώδη δικαιώματα και τηρεί τις ελευθερίες και τις αρχές που αναγνωρίζονται στον Χάρτη, όπως ενσωματώνονται στις Συνθήκες, ιδίως το σεβασμό της ιδιωτικής και οικογενειακής ζωής, της κατοικίας και της επικοινωνίας, την προστασία των προσωπικών δεδομένων, την ελευθερία σκέψης, συνείδησης και θρησκείας, την ελευθερία έκφρασης και πληροφόρησης, την ελευθερία άσκησης επιχειρηματικής δραστηριότητας, το δικαίωμα σε αποτελεσματική προσφυγή και δίκαιη δίκη και την πολιτιστική, θρησκευτική και γλωσσική πολυμορφία. Το δικαίωμα στην προστασία των προσωπικών δεδομένων πρέπει να εξετάζεται σε σχέση με τη λειτουργία του στην κοινωνία και να εξισορροπείται με άλλα θεμελιώδη δικαιώματα, σύμφωνα με την αρχή της αναλογικότητας."
},
{
    'law': 'ai_act',
    'question': "Πώς προτείνει ο Νόμος για την Τεχνητή Νοημοσύνη να ρυθμίζει τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη ταξινομεί τα συστήματα τεχνητής νοημοσύνης με βάση τον κίνδυνο που ενέχουν και υπόκεινται σε αυστηρές απαιτήσεις τα συστήματα υψηλού κινδύνου. Τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου περιλαμβάνουν αυτά που χρησιμοποιούνται σε κρίσιμες υποδομές, εκπαίδευση, απασχόληση, βασικές δημόσιες και ιδιωτικές υπηρεσίες, επιβολή του νόμου, και διαχείριση της μετανάστευσης, του ασύλου και των συνόρων. Αυτά τα συστήματα πρέπει να συμμορφώνονται με απαιτήσεις που σχετίζονται με τη διαχείριση κινδύνων, τη διακυβέρνηση δεδομένων, την τεχνική τεκμηρίωση, την τήρηση αρχείων, τη διαφάνεια, την παροχή πληροφοριών στους χρήστες, την ανθρώπινη επίβλεψη, την ακρίβεια και την ανθεκτικότητα. Οι πάροχοι αυτών των συστημάτων πρέπει να καθιερώσουν ένα σύστημα διαχείρισης ποιότητας και να εξασφαλίσουν συνεχή παρακολούθηση και εποπτεία μετά την κυκλοφορία στην αγορά."
},
{
    'law': 'dma',
    'question': "Πώς προτείνει ο DMA να ρυθμίζει τη συμπεριφορά των διαμεσολαβητών στις ψηφιακές αγορές;",
    'answer': "Ο DMA επιβάλλει συγκεκριμένες υποχρεώσεις στους διαμεσολαβητές για να τους αποτρέψει από το να εμπλέκονται σε άδικες πρακτικές που βλάπτουν τον ανταγωνισμό και τους καταναλωτές. Αυτό περιλαμβάνει την απαγόρευση στους διαμεσολαβητές να ευνοούν τις δικές τους υπηρεσίες έναντι των ανταγωνιστών (αυτοπροτίμηση), απαιτώντας από αυτούς να επιτρέπουν τη διαλειτουργικότητα με υπηρεσίες τρίτων και να διασφαλίζουν ότι δεν περιορίζουν αθέμιτα την πρόσβαση στις πλατφόρμες τους. Οι διαμεσολαβητές υποχρεούνται επίσης να παρέχουν φορητότητα δεδομένων, να προσφέρουν δίκαιους όρους στους επιχειρηματικούς χρήστες και να εξασφαλίζουν διαφάνεια στις λειτουργίες τους."
},
{
    'law': 'dsa',
    'question': "Πώς προτείνει ο DSA να προστατεύσει τους χρήστες από παράνομο περιεχόμενο σε ψηφιακές πλατφόρμες;",
    'answer': "Ο DSA στοχεύει στην προστασία των χρηστών από παράνομο περιεχόμενο απαιτώντας από τις πλατφόρμες να εφαρμόσουν μηχανισμούς ειδοποίησης και δράσης, επιτρέποντας στους χρήστες να αναφέρουν εύκολα παράνομο περιεχόμενο. Οι πλατφόρμες πρέπει να ενεργούν άμεσα για να αφαιρέσουν ή να απενεργοποιήσουν την πρόσβαση σε παράνομο περιεχόμενο μόλις λάβουν ειδοποίηση. Ο DSA εισάγει επίσης υποχρεώσεις για τις πλατφόρμες να συνεργάζονται με τις αρχές επιβολής του νόμου και να παρέχουν εκθέσεις διαφάνειας για τις δραστηριότητες εποπτείας περιεχομένου τους. Οι πλατφόρμες πρέπει να λαμβάνουν προληπτικά μέτρα για να αποτρέψουν τη διάδοση παράνομου περιεχομένου και να διασφαλίζουν ότι οι αλγόριθμοί τους δεν προωθούν επιβλαβές ή παράνομο περιεχόμενο."
},
{
    'law': 'gdpr',
    'question': "Ποια προβλήματα έχουν προκύψει λόγω τεχνολογικών εξελίξεων και παγκοσμιοποίησης στο πλαίσιο της προστασίας δεδομένων προσωπικού χαρακτήρα;",
    'answer': "Οι τεχνολογικές εξελίξεις και η παγκοσμιοποίηση έχουν δημιουργήσει νέες προκλήσεις για την προστασία των προσωπικών δεδομένων. Η κλίμακα συλλογής και κοινοποίησης δεδομένων προσωπικού χαρακτήρα έχει αυξηθεί σημαντικά. Η τεχνολογία επιτρέπει σε ιδιωτικές εταιρείες και δημόσιες αρχές να χρησιμοποιούν προσωπικά δεδομένα σε πρωτοφανή κλίμακα για την επίτευξη των δραστηριοτήτων τους. Τα φυσικά πρόσωπα δημοσιοποιούν ολοένα και περισσότερο προσωπικές πληροφορίες σε δημόσια και παγκόσμια κλίμακα. Η τεχνολογία έχει μεταμορφώσει τόσο την οικονομία όσο και την κοινωνική ζωή, και θα πρέπει να διευκολύνει περαιτέρω την ελεύθερη ροή προσωπικών δεδομένων εντός της Ένωσης και τη μεταφορά σε τρίτες χώρες και διεθνείς οργανισμούς, διασφαλίζοντας παράλληλα ένα υψηλό επίπεδο προστασίας των προσωπικών δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιες ευθύνες επιβάλλει ο Νόμος για την Τεχνητή Νοημοσύνη στους παρόχους τεχνητής νοημοσύνης για να διασφαλιστούν ηθικές πρακτικές;",
    'answer': "Οι πάροχοι συστημάτων τεχνητής νοημοσύνης υψηλού κινδύνου είναι υπεύθυνοι να διασφαλίζουν ότι τα συστήματά τους συμμορφώνονται με τις απαιτήσεις που ορίζει ο νόμος. Αυτό περιλαμβάνει την υποχρέωση διενέργειας αξιολόγησης συμμόρφωσης πριν από την τοποθέτηση του συστήματος στην αγορά, να διασφαλίζεται ότι το σύστημα υποβάλλεται σε κατάλληλες δοκιμές, να παρέχονται σαφείς οδηγίες και πληροφορίες στους χρήστες, να εφαρμόζονται μέτρα ανθρώπινης επίβλεψης και να παρακολουθείται το σύστημα καθ’ όλη τη διάρκεια της ζωής του. Οι πάροχοι πρέπει επίσης να αναφέρουν σοβαρά περιστατικά και δυσλειτουργίες στις αρμόδιες αρχές."
},
{
    'law': 'dma',
    'question': "Ποιες είναι οι βασικές υποχρεώσεις που επιβάλλει ο DMA στους διαμεσολαβητές;",
    'answer': "Οι βασικές υποχρεώσεις για τους διαμεσολαβητές βάσει του DMA περιλαμβάνουν απαγορεύσεις στη συνδυαστική χρήση προσωπικών δεδομένων από διαφορετικές πηγές χωρίς τη συγκατάθεση του χρήστη, περιορισμούς στην προεγκατάσταση λογισμικού ή εφαρμογών, και απαιτήσεις να επιτρέπουν στους επιχειρηματικούς χρήστες την πρόσβαση στα δεδομένα που δημιουργούνται στην πλατφόρμα τους. Οι διαμεσολαβητές πρέπει επίσης να διασφαλίζουν ότι οι πλατφόρμες τους είναι ανοιχτές και διαλειτουργικές με υπηρεσίες τρίτων και απαγορεύεται να χρησιμοποιούν μη δημόσια δεδομένα από τους επιχειρηματικούς χρήστες για να ανταγωνίζονται εναντίον τους."
},
{
    'law': 'dsa',
    'question': "Ποιες απαιτήσεις διαφάνειας επιβάλλονται στις διαδικτυακές πλατφόρμες από τον DSA;",
    'answer': "Ο DSA επιβάλλει εκτεταμένες απαιτήσεις διαφάνειας στις διαδικτυακές πλατφόρμες, περιλαμβάνοντας την υποχρέωση δημοσίευσης εκθέσεων διαφάνειας που αναλύουν τον αριθμό των ενεργειών αφαίρεσης περιεχομένου, τους λόγους για αυτές τις ενέργειες και τα αποτελέσματα των ενστάσεων των χρηστών. Οι πλατφόρμες πρέπει επίσης να αποκαλύπτουν πώς λειτουργούν τα συστήματα εποπτείας περιεχομένου και οι αλγόριθμοι σύστασης, περιλαμβάνοντας τα κριτήρια που χρησιμοποιούνται για την κατάταξη και την προβολή περιεχομένου. Οι χρήστες πρέπει να ενημερώνονται για τους όρους και τις προϋποθέσεις που διέπουν τη χρήση της πλατφόρμας και για οποιεσδήποτε αλλαγές γίνονται σε αυτούς τους όρους. Επιπλέον, οι πλατφόρμες πρέπει να παρέχουν σαφείς πληροφορίες για τις διαφημίσεις που προβάλλουν, περιλαμβάνοντας την ταυτότητα των διαφημιστών και τα κριτήρια στόχευσης."
},
{
    'law': 'gdpr',
    'question': "Πώς αντιμετωπίζει ο GDPR τη μεταφορά δεδομένων προσωπικού χαρακτήρα σε τρίτες χώρες ή διεθνείς οργανισμούς;",
    'answer': "Η μεταφορά δεδομένων προσωπικού χαρακτήρα σε τρίτες χώρες ή διεθνείς οργανισμούς επιτρέπεται μόνο όταν πληρούνται οι προϋποθέσεις που ορίζονται στον παρόντα Κανονισμό, ώστε να διασφαλιστεί ότι το επίπεδο προστασίας των φυσικών προσώπων που εγγυάται ο Κανονισμός αυτός δεν υπονομεύεται. Σε κάθε περίπτωση, οι μεταφορές σε τρίτες χώρες και διεθνείς οργανισμούς μπορούν να πραγματοποιούνται μόνο με πλήρη συμμόρφωση με τον παρόντα Κανονισμό. Ο παρών Κανονισμός δεν θίγει τις διεθνείς συμφωνίες που έχουν συναφθεί μεταξύ της Ένωσης και τρίτων χωρών που ρυθμίζουν τη μεταφορά δεδομένων προσωπικού χαρακτήρα, συμπεριλαμβανομένων των κατάλληλων διασφαλίσεων για τα υποκείμενα των δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Πώς αντιμετωπίζει ο Νόμος για την Τεχνητή Νοημοσύνη τη διαφάνεια και τη λογοδοσία στα συστήματα τεχνητής νοημοσύνης;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη απαιτεί τα συστήματα τεχνητής νοημοσύνης, ιδιαίτερα τα υψηλού κινδύνου, να είναι διαφανή και να παρέχουν σαφείς πληροφορίες σχετικά με τον σκοπό, τις δυνατότητες και τους περιορισμούς τους. Οι χρήστες πρέπει να μπορούν να κατανοήσουν πώς λαμβάνονται οι αποφάσεις από τα συστήματα τεχνητής νοημοσύνης και ποια δεδομένα επεξεργάζονται. Ο νόμος απαιτεί τα συστήματα τεχνητής νοημοσύνης να σχεδιάζονται με χαρακτηριστικά που διασφαλίζουν τη λογοδοσία, συμπεριλαμβανομένης της δυνατότητας ελέγχου, της ανιχνευσιμότητας των αποφάσεων και της δυνατότητας παροχής εξηγήσεων για τις αποφάσεις που λαμβάνονται από την τεχνητή νοημοσύνη."
},
{
    'law': 'dma',
    'question': "Πώς στοχεύει ο DMA να αποτρέψει τις άδικες πρακτικές στην ψηφιακή αγορά;",
    'answer': "Ο DMA στοχεύει στην αποτροπή των άδικων πρακτικών θέτοντας σαφείς κανόνες για τους διαμεσολαβητές, συμπεριλαμβανομένων των απαγορεύσεων στην αυτοπροτίμηση, των περιορισμών στους άδικους όρους και τις προϋποθέσεις για τους επιχειρηματικούς χρήστες, και των απαιτήσεων για διαφάνεια στον τρόπο λειτουργίας τους. Ο DMA εξασφαλίζει επίσης ότι οι διαμεσολαβητές δεν μπορούν να χρησιμοποιήσουν τη δεσπόζουσα θέση τους για να περιορίσουν τον ανταγωνισμό ή την καινοτομία από μικρότερες επιχειρήσεις. Η Ευρωπαϊκή Επιτροπή έχει εξουσία να ερευνά και να επιβάλλει κυρώσεις σε διαμεσολαβητές που δεν συμμορφώνονται με αυτούς τους κανόνες."
},
{
    'law': 'dsa',
    'question': "Πώς προτείνει ο DSA να αντιμετωπιστεί η διάδοση επιβλαβούς περιεχομένου;",
    'answer': "Ο DSA προτείνει να αντιμετωπιστεί η διάδοση επιβλαβούς περιεχομένου απαιτώντας από τις πλατφόρμες να αξιολογούν τους κινδύνους που σχετίζονται με τη διάδοση επιβλαβούς ή παράνομου περιεχομένου και να λαμβάνουν κατάλληλα μέτρα για να μετριάσουν αυτούς τους κινδύνους. Οι πλατφόρμες πρέπει να εφαρμόζουν προστατευτικά μέτρα για να διασφαλίσουν ότι οι αλγόριθμοί τους δεν προωθούν επιβλαβές περιεχόμενο και να παρέχουν στους χρήστες εργαλεία για να ελέγχουν το περιεχόμενο στο οποίο εκτίθενται. Ο DSA ενθαρρύνει επίσης τις πλατφόρμες να συνεργάζονται με αξιόπιστους ειδοποιητές και ελεγκτές γεγονότων για να εντοπίζουν και να αντιμετωπίζουν πιο αποτελεσματικά το επιβλαβές περιεχόμενο. Σε περιπτώσεις όπου οι πλατφόρμες αποτυγχάνουν να μετριάσουν επαρκώς τους κινδύνους, μπορεί να υπόκεινται σε κανονιστικές ενέργειες, περιλαμβάνοντας πρόστιμα και άλλες κυρώσεις."
},
{
    'law': 'gdpr',
    'question': "Ποιες συγκεκριμένες προστασίες προσφέρει ο GDPR στα παιδιά σχετικά με τα προσωπικά τους δεδομένα;",
    'answer': "Τα παιδιά αξίζουν ειδική προστασία όσον αφορά τα προσωπικά τους δεδομένα, καθώς μπορεί να μην γνωρίζουν επαρκώς τους κινδύνους, τις συνέπειες, τις διασφαλίσεις και τα δικαιώματα σε σχέση με την επεξεργασία δεδομένων προσωπικού χαρακτήρα. Αυτή η ειδική προστασία θα πρέπει, ιδίως, να ισχύει για τη χρήση προσωπικών δεδομένων παιδιών για σκοπούς μάρκετινγκ ή δημιουργίας προφίλ προσωπικότητας ή χρήστη και για τη συλλογή δεδομένων προσωπικού χαρακτήρα σε σχέση με παιδιά κατά τη χρήση υπηρεσιών που προσφέρονται απευθείας σε ένα παιδί. Η συγκατάθεση του κατόχου της γονικής ευθύνης δεν θα πρέπει να είναι απαραίτητη στο πλαίσιο προληπτικών ή συμβουλευτικών υπηρεσιών που προσφέρονται απευθείας σε ένα παιδί."
},
{
    'law': 'ai_act',
    'question': "Ποια μέτρα προτείνει ο Νόμος για την Τεχνητή Νοημοσύνη για την προστασία των θεμελιωδών δικαιωμάτων κατά την ανάπτυξη τεχνολογιών τεχνητής νοημοσύνης;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη περιλαμβάνει αρκετά μέτρα για την προστασία των θεμελιωδών δικαιωμάτων, όπως η απαίτηση να σχεδιάζονται και να χρησιμοποιούνται τα συστήματα τεχνητής νοημοσύνης με τρόπο που να είναι συνεπής με τον σεβασμό της ανθρώπινης αξιοπρέπειας, της ιδιωτικότητας, της μη διάκρισης και άλλων θεμελιωδών δικαιωμάτων. Αυτό περιλαμβάνει την ενσωμάτωση μηχανισμών ανθρώπινης επίβλεψης, τη διασφάλιση ότι τα συστήματα τεχνητής νοημοσύνης δεν οδηγούν σε προκατειλημμένα ή διακριτικά αποτελέσματα και την παροχή τρόπων για τα άτομα να αμφισβητήσουν αποφάσεις που λαμβάνονται από συστήματα τεχνητής νοημοσύνης που τους επηρεάζουν σημαντικά. Ο νόμος προωθεί επίσης την ανάπτυξη κωδίκων δεοντολογίας και εθελοντικών μέτρων από τους παρόχους για να διασφαλιστεί ότι η τεχνητή νοημοσύνη χρησιμοποιείται ηθικά και σε ευθυγράμμιση με τις αξίες της κοινωνίας."
},
{
    'law': 'dma',
    'question': "Ποιοι μηχανισμοί επιβολής περιλαμβάνονται στον DMA για να διασφαλίσουν τη συμμόρφωση των διαμεσολαβητών;",
    'answer': "Ο DMA περιλαμβάνει ισχυρούς μηχανισμούς επιβολής, όπως τη δυνατότητα της Ευρωπαϊκής Επιτροπής να επιβάλλει πρόστιμα έως και 10% του συνολικού παγκόσμιου ετήσιου κύκλου εργασιών του διαμεσολαβητή για μη συμμόρφωση. Σε περιπτώσεις επανειλημμένων παραβάσεων, η Επιτροπή μπορεί να επιβάλλει επιπλέον κυρώσεις, όπως διαρθρωτικά μέτρα, όπως η διάθεση επιχειρήσεων. Ο DMA επιτρέπει επίσης περιοδικές κυρώσεις πληρωμών για να διασφαλίσουν ότι οι διαμεσολαβητές συμμορφώνονται με τις υποχρεώσεις και τις απαγορεύσεις που ορίζονται στον κανονισμό."
},
{
    'law': 'dsa',
    'question': "Ποια μέτρα περιλαμβάνει ο DSA για την προστασία της ελευθερίας της έκφρασης ενώ καταπολεμά το παράνομο περιεχόμενο;",
    'answer': "Ο DSA περιλαμβάνει μέτρα για την προστασία της ελευθερίας της έκφρασης διασφαλίζοντας ότι οποιοιδήποτε περιορισμοί στο περιεχόμενο είναι αναγκαίοι, αναλογικοί και νομικά δικαιολογημένοι. Οι πλατφόρμες πρέπει να παρέχουν στους χρήστες σαφείς εξηγήσεις όταν το περιεχόμενο αφαιρείται ή η πρόσβαση περιορίζεται, και οι χρήστες πρέπει να έχουν το δικαίωμα να υποβάλουν ένσταση κατά αυτών των αποφάσεων. Ο DSA απαιτεί επίσης από τις πλατφόρμες να διασφαλίζουν ότι οι διαδικασίες εποπτείας περιεχομένου είναι δίκαιες και διαφανείς, με διασφαλίσεις για την αποφυγή της αυθαίρετης αφαίρεσης περιεχομένου. Επιπλέον, ο DSA ενθαρρύνει τις πλατφόρμες να αναπτύξουν κώδικες δεοντολογίας σε συνεργασία με τους ενδιαφερόμενους φορείς για να ισορροπήσουν την ανάγκη καταπολέμησης του παράνομου περιεχομένου με την προστασία της ελευθερίας του λόγου."
},
{
    'law': 'gdpr',
    'question': "Πώς ορίζει ο GDPR τα προσωπικά δεδομένα και ποια είναι μερικά παραδείγματα;",
    'answer': "Τα προσωπικά δεδομένα βάσει του GDPR ορίζονται ως οποιαδήποτε πληροφορία που σχετίζεται με ένα ταυτοποιημένο ή ταυτοποιήσιμο φυσικό πρόσωπο ('υποκείμενο των δεδομένων'). Παραδείγματα περιλαμβάνουν το όνομα ενός ατόμου, τον αριθμό ταυτοποίησης, δεδομένα τοποθεσίας, το αναγνωριστικό στο διαδίκτυο, ή έναν ή περισσότερους παράγοντες που αφορούν τη φυσική, φυσιολογική, γενετική, ψυχική, οικονομική, πολιτιστική ή κοινωνική ταυτότητα του φυσικού προσώπου. Ο ορισμός είναι ευρύς, καλύπτοντας διάφορες μορφές δεδομένων που θα μπορούσαν να χρησιμοποιηθούν για την άμεση ή έμμεση ταυτοποίηση ενός ατόμου."
},
{
    'law': 'ai_act',
    'question': "Πώς ορίζει ο Νόμος για την Τεχνητή Νοημοσύνη το 'σύστημα τεχνητής νοημοσύνης' και ποιες τεχνολογίες εμπίπτουν σε αυτόν τον ορισμό;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη ορίζει ένα 'σύστημα τεχνητής νοημοσύνης' ως λογισμικό που αναπτύσσεται με μία ή περισσότερες από τις τεχνικές και προσεγγίσεις που απαριθμούνται στον νόμο, όπως η μηχανική μάθηση, οι λογικές και γνωστικές προσεγγίσεις και οι στατιστικές προσεγγίσεις. Αυτά τα συστήματα μπορούν, για ένα καθορισμένο σύνολο ανθρώπινων στόχων, να παράγουν αποτελέσματα όπως περιεχόμενο, προβλέψεις, συστάσεις ή αποφάσεις που επηρεάζουν τα περιβάλλοντα με τα οποία αλληλεπιδρούν. Ο ορισμός είναι ευρύς και περιλαμβάνει μια ποικιλία τεχνολογιών τεχνητής νοημοσύνης, από απλούς αλγόριθμους μέχρι σύνθετα μοντέλα μηχανικής μάθησης."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα της αυτοπροτίμησης από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαγορεύει ειδικά στους διαμεσολαβητές να εμπλέκονται σε πρακτικές αυτοπροτίμησης, όπου ευνοούν τα δικά τους προϊόντα ή υπηρεσίες έναντι αυτών των ανταγωνιστών στις πλατφόρμες τους. Αυτό περιλαμβάνει πρακτικές όπως η κατάταξη των δικών τους προϊόντων ψηλότερα στα αποτελέσματα αναζήτησης ή η παροχή προνομιακής πρόσβασης σε δεδομένα. Ο στόχος είναι να διασφαλιστεί ένα ίσο επίπεδο ανταγωνισμού στις ψηφιακές αγορές, όπου ο ανταγωνισμός βασίζεται στην αξία και όχι στη δύναμη της αγοράς του διαμεσολαβητή. Η απαγόρευση της αυτοπροτίμησης είναι μία από τις βασικές υποχρεώσεις που επιβάλλονται στους διαμεσολαβητές για να αποτραπεί η αντι-ανταγωνιστική συμπεριφορά."
},
{
    'law': 'dsa',
    'question': "Πώς αντιμετωπίζει ο DSA το ζήτημα της εποπτείας περιεχομένου στις διαδικτυακές πλατφόρμες;",
    'answer': "Ο DSA απαιτεί από τις διαδικτυακές πλατφόρμες να εφαρμόζουν πολιτικές εποπτείας περιεχομένου που είναι διαφανείς, συνεπείς και ευθυγραμμισμένες με τα θεμελιώδη δικαιώματα. Οι πλατφόρμες πρέπει να θεσπίσουν σαφείς όρους και προϋποθέσεις για την εποπτεία περιεχομένου και να παρέχουν στους χρήστες λεπτομερείς πληροφορίες για το πώς αξιολογείται, αφαιρείται ή περιορίζεται το περιεχόμενο. Ο DSA επίσης επιβάλλει στις πλατφόρμες να εφαρμόζουν μηχανισμούς για τους χρήστες ώστε να υποβάλουν ένσταση κατά αποφάσεων εποπτείας περιεχομένου, διασφαλίζοντας ότι οι χρήστες έχουν την ευκαιρία να αντιταχθούν σε αδικαιολόγητες αφαιρέσεις ή περιορισμούς. Αυτά τα μέτρα στοχεύουν στη δημιουργία ενός δίκαιου και υπεύθυνου συστήματος εποπτείας περιεχομένου που σέβεται την ελευθερία της έκφρασης ενώ καταπολεμά το παράνομο περιεχόμενο."
},
{
    'law': 'gdpr',
    'question': "Ποια είναι η νομική βάση για την επεξεργασία προσωπικών δεδομένων σύμφωνα με τον GDPR;",
    'answer': "Ο GDPR περιγράφει διάφορες νομικές βάσεις για την επεξεργασία προσωπικών δεδομένων, συμπεριλαμβανομένων: το υποκείμενο των δεδομένων έχει δώσει συγκατάθεση για την επεξεργασία· η επεξεργασία είναι απαραίτητη για την εκτέλεση μιας σύμβασης στην οποία το υποκείμενο των δεδομένων είναι μέρος· η επεξεργασία είναι απαραίτητη για τη συμμόρφωση με μια νομική υποχρέωση· η επεξεργασία είναι απαραίτητη για την προστασία των ζωτικών συμφερόντων του υποκειμένου των δεδομένων ή άλλου φυσικού προσώπου· η επεξεργασία είναι απαραίτητη για την εκτέλεση καθήκοντος που εκτελείται προς το δημόσιο συμφέρον ή την άσκηση επίσημης αρχής· και η επεξεργασία είναι απαραίτητη για τους σκοπούς των έννομων συμφερόντων που επιδιώκονται από τον υπεύθυνο επεξεργασίας ή τρίτο μέρος, εκτός εάν αυτά τα συμφέροντα υπερισχύονται από τα συμφέροντα ή τα θεμελιώδη δικαιώματα και ελευθερίες του υποκειμένου των δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιες κατηγορίες συστημάτων τεχνητής νοημοσύνης θεωρούνται υψηλού κινδύνου βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου βάσει του Νόμου για την Τεχνητή Νοημοσύνη περιλαμβάνουν αυτά που χρησιμοποιούνται σε κρίσιμες υποδομές (όπως μεταφορές, ενέργεια και ύδρευση), εκπαίδευση και επαγγελματική κατάρτιση, διαχείριση της απασχόλησης και των εργαζομένων, πρόσβαση σε βασικές ιδιωτικές και δημόσιες υπηρεσίες (όπως πιστωτική αξιολόγηση και κοινωνικά επιδόματα), επιβολή του νόμου (όπως προληπτική αστυνόμευση), διαχείριση της μετανάστευσης, του ασύλου και των συνόρων, και τη διοίκηση της δικαιοσύνης και των δημοκρατικών διαδικασιών. Αυτά τα συστήματα υπόκεινται σε αυστηρές απαιτήσεις λόγω των σημαντικών κινδύνων που θέτουν στα θεμελιώδη δικαιώματα και την ασφάλεια."
},
{
    'law': 'dma',
    'question': "Ποια είναι τα κριτήρια για την αναγνώριση των βασικών υπηρεσιών πλατφόρμας βάσει του DMA;",
    'answer': "Οι βασικές υπηρεσίες πλατφόρμας βάσει του DMA περιλαμβάνουν μια σειρά ψηφιακών υπηρεσιών που λειτουργούν ως σημαντικές πύλες για τους επιχειρηματικούς χρήστες για να φτάσουν στους τελικούς χρήστες. Αυτές οι υπηρεσίες περιλαμβάνουν υπηρεσίες διαδικτυακής διαμεσολάβησης, όπως καταστήματα εφαρμογών και αγορές, μηχανές αναζήτησης στο διαδίκτυο, υπηρεσίες κοινωνικής δικτύωσης, υπηρεσίες πλατφορμών κοινής χρήσης βίντεο, υπηρεσίες ανεξάρτητης επικοινωνίας, λειτουργικά συστήματα, υπηρεσίες υπολογιστικού νέφους και υπηρεσίες διαφήμισης. Μια υπηρεσία θεωρείται βασική υπηρεσία πλατφόρμας εάν έχει σημαντική επίδραση στην εσωτερική αγορά και είναι μια ουσιαστική πύλη για τους επιχειρηματικούς χρήστες να έχουν πρόσβαση στους τελικούς χρήστες."
},
{
    'law': 'dsa',
    'question': "Ποιες υποχρεώσεις έχουν οι πολύ μεγάλες διαδικτυακές πλατφόρμες (VLOPs) βάσει του DSA;",
    'answer': "Οι πολύ μεγάλες διαδικτυακές πλατφόρμες (VLOPs), ορίζονται ως πλατφόρμες με περισσότερους από 45 εκατομμύρια χρήστες στην ΕΕ, έχουν πρόσθετες υποχρεώσεις βάσει του DSA λόγω της σημαντικής τους επίδρασης στην κοινωνία και τον δημόσιο διάλογο. Οι VLOPs πρέπει να διεξάγουν ετήσιες αξιολογήσεις κινδύνου για τον εντοπισμό και τη μείωση των συστημικών κινδύνων, όπως η διάδοση παράνομου περιεχομένου, παραπληροφόρησης και επιβλαβούς περιεχομένου. Πρέπει επίσης να παρέχουν μεγαλύτερη διαφάνεια στους αλγόριθμους σύστασης περιεχομένου τους, να προσφέρουν στους χρήστες μεγαλύτερο έλεγχο στο περιεχόμενο που βλέπουν και να συνεργάζονται με τις αρχές για την πρόληψη και αντιμετώπιση των συστημικών κινδύνων. Αυτές οι υποχρεώσεις έχουν σχεδιαστεί για να διασφαλίσουν ότι οι VLOPs λειτουργούν με τρόπο που είναι ασφαλής, διαφανής και σεβαστός στα θεμελιώδη δικαιώματα."
},
{
    'law': 'gdpr',
    'question': "Ποια είναι τα δικαιώματα των υποκειμένων των δεδομένων σύμφωνα με τον GDPR;",
    'answer': "Ο GDPR παρέχει στα υποκείμενα των δεδομένων διάφορα δικαιώματα, συμπεριλαμβανομένου του δικαιώματος ενημέρωσης, του δικαιώματος πρόσβασης, του δικαιώματος διόρθωσης, του δικαιώματος διαγραφής ('δικαίωμα στη λήθη'), του δικαιώματος περιορισμού της επεξεργασίας, του δικαιώματος φορητότητας των δεδομένων, του δικαιώματος αντίρρησης στην επεξεργασία και δικαιώματα που σχετίζονται με την αυτοματοποιημένη λήψη αποφάσεων και την κατάρτιση προφίλ. Αυτά τα δικαιώματα δίνουν τη δυνατότητα στα άτομα να έχουν έλεγχο στα προσωπικά τους δεδομένα και να διασφαλίζουν τη διαφάνεια και τη λογοδοσία στην επεξεργασία δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιες υποχρεώσεις έχουν οι χρήστες συστημάτων τεχνητής νοημοσύνης υψηλού κινδύνου βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Οι χρήστες συστημάτων τεχνητής νοημοσύνης υψηλού κινδύνου απαιτείται να λειτουργούν τα συστήματα σύμφωνα με τις οδηγίες που παρέχει ο πάροχος του συστήματος τεχνητής νοημοσύνης, να παρακολουθούν τη λειτουργία του συστήματος και να αναφέρουν αμέσως οποιαδήποτε σοβαρά περιστατικά ή δυσλειτουργίες στον πάροχο και τις αρμόδιες αρχές. Οι χρήστες πρέπει επίσης να διατηρούν αρχεία που δημιουργούνται από το σύστημα τεχνητής νοημοσύνης, να διασφαλίζουν ότι διατηρείται η ανθρώπινη επίβλεψη και να διασφαλίζουν ότι το σύστημα τεχνητής νοημοσύνης χρησιμοποιείται μόνο για τον προορισμένο σκοπό του. Επιπλέον, οι χρήστες είναι υπεύθυνοι για την εφαρμογή μέτρων για τον μετριασμό των κινδύνων για τα θεμελιώδη δικαιώματα και την ασφάλεια."
},
{
    'law': 'dma',
    'question': "Πώς προάγει ο DMA τη διαλειτουργικότητα μεταξύ ψηφιακών υπηρεσιών;",
    'answer': "Ο DMA προάγει τη διαλειτουργικότητα απαιτώντας από τους διαμεσολαβητές να διασφαλίζουν ότι οι βασικές υπηρεσίες πλατφόρμας τους μπορούν να αλληλεπιδρούν με υπηρεσίες τρίτων. Αυτό περιλαμβάνει την παροχή των απαραίτητων τεχνικών διεπαφών και τεκμηρίωσης για να επιτρέπεται η διαλειτουργικότητα. Ο στόχος είναι να αποτραπεί η κλειδωματική χρήση των πλατφορμών από τους χρήστες και τους επιχειρηματικούς χρήστες και να επιτραπεί ο ανταγωνισμός επιτρέποντας νέες είσοδοι και μικρότεροι ανταγωνιστές να προσφέρουν συμπληρωματικές ή ανταγωνιστικές υπηρεσίες. Η διαλειτουργικότητα θεωρείται ως ένα βασικό μέτρο για την προώθηση της καινοτομίας και της επιλογής καταναλωτών στις ψηφιακές αγορές."
},
{
    'law': 'dsa',
    'question': "Πώς ενισχύει ο DSA την προστασία των ανηλίκων στο διαδίκτυο;",
    'answer': "Ο DSA περιλαμβάνει συγκεκριμένες διατάξεις για την ενίσχυση της προστασίας των ανηλίκων στο διαδίκτυο, αναγνωρίζοντας ότι τα παιδιά είναι ιδιαίτερα ευάλωτα σε επιβλαβές περιεχόμενο και πρακτικές. Οι πλατφόρμες πρέπει να εφαρμόζουν μέτρα για να διασφαλίσουν ότι οι υπηρεσίες τους είναι ασφαλείς για ανηλίκους, περιλαμβάνοντας την εποπτεία περιεχομένου που είναι κατάλληλη για την ηλικία, τον γονικό έλεγχο και τους περιορισμούς στις στοχευμένες διαφημίσεις προς ανηλίκους. Ο DSA απαιτεί επίσης από τις πλατφόρμες να παρέχουν σαφείς και προσιτές πληροφορίες στους ανηλίκους και στους γονείς τους σχετικά με τους κινδύνους που συνδέονται με τις διαδικτυακές δραστηριότητες και πώς να προστατευτούν. Αυτά τα μέτρα αποσκοπούν στη δημιουργία ενός ασφαλέστερου διαδικτυακού περιβάλλοντος για τα παιδιά και στην ενδυνάμωσή τους και των κηδεμόνων τους να λαμβάνουν ενημερωμένες αποφάσεις."
},
{
    'law': 'gdpr',
    'question': "Πώς αντιμετωπίζει ο GDPR την προστασία των δεδομένων εξ ορισμού και εξ σχεδιασμού;",
    'answer': "Ο GDPR απαιτεί από τους υπεύθυνους επεξεργασίας να εφαρμόζουν την προστασία δεδομένων εξ ορισμού και εξ σχεδιασμού. Αυτό σημαίνει ότι τα μέτρα προστασίας των δεδομένων πρέπει να ενσωματώνονται στις δραστηριότητες επεξεργασίας από την αρχή και ότι μόνο τα απαραίτητα προσωπικά δεδομένα για κάθε συγκεκριμένο σκοπό επεξεργασίας επεξεργάζονται. Ο υπεύθυνος επεξεργασίας πρέπει να λάβει τα κατάλληλα τεχνικά και οργανωτικά μέτρα, όπως η ψευδωνυμοποίηση, για να διασφαλίσει ότι, εξ ορισμού, τα προσωπικά δεδομένα δεν είναι προσβάσιμα σε αόριστο αριθμό ατόμων χωρίς τη συγκατάθεση του ατόμου."
},
{
    'law': 'ai_act',
    'question': "Πώς αντιμετωπίζει ο Νόμος για την Τεχνητή Νοημοσύνη τη χρήση συστημάτων βιομετρικής αναγνώρισης;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη επιβάλλει αυστηρούς κανονισμούς για τη χρήση συστημάτων βιομετρικής αναγνώρισης, ιδίως εκείνων που χρησιμοποιούνται σε δημόσιους χώρους για σκοπούς επιβολής του νόμου. Η χρήση συστημάτων βιομετρικής αναγνώρισης σε πραγματικό χρόνο σε δημόσιους προσβάσιμους χώρους γενικά απαγορεύεται, με εξαιρέσεις που χορηγούνται υπό συγκεκριμένες συνθήκες, όπως η αποτροπή μιας τρομοκρατικής επίθεσης, ο εντοπισμός ενός αγνοούμενου παιδιού ή η ταυτοποίηση ενός υπόπτου για σοβαρό έγκλημα. Ακόμα και σε αυτές τις περιπτώσεις, η χρήση πρέπει να εγκρίνεται από δικαστικές ή άλλες ανεξάρτητες αρχές και να υπόκειται σε αυστηρές διασφαλίσεις για την προστασία των θεμελιωδών δικαιωμάτων."
},
{
    'law': 'dma',
    'question': "Ποιες υποχρεώσεις επιβάλλει ο DMA στους διαμεσολαβητές σχετικά με την πρόσβαση στα δεδομένα και τη φορητότητα;",
    'answer': "Ο DMA επιβάλλει υποχρεώσεις στους διαμεσολαβητές να παρέχουν στους επιχειρηματικούς χρήστες και τους τελικούς χρήστες πρόσβαση στα δεδομένα που δημιουργούνται μέσω των αλληλεπιδράσεών τους στην πλατφόρμα. Αυτό περιλαμβάνει την παροχή δεδομένων σε μια δομημένη, κοινή χρήση, και μηχανικά αναγνώσιμη μορφή για να διευκολύνεται η φορητότητα δεδομένων. Οι διαμεσολαβητές είναι επίσης υποχρεωμένοι να επιτρέπουν στους επιχειρηματικούς χρήστες να έχουν πρόσβαση σε δεδομένα που είναι απαραίτητα για την ανάπτυξη και βελτίωση των δικών τους προϊόντων και υπηρεσιών. Αυτές οι υποχρεώσεις αποσκοπούν στο να αποτρέψουν τους διαμεσολαβητές από το να χρησιμοποιούν τον έλεγχό τους επί των δεδομένων για να περιορίσουν τον ανταγωνισμό και την καινοτομία."
},
{
    'law': 'dsa',
    'question': "Ποιες είναι οι υποχρεώσεις διαφάνειας για τις διαδικτυακές πλατφόρμες όσον αφορά τους αλγόριθμούς τους;",
    'answer': "Ο DSA επιβάλλει υποχρεώσεις διαφάνειας στις διαδικτυακές πλατφόρμες να παρέχουν σαφείς και προσιτές πληροφορίες για το πώς λειτουργούν οι αλγόριθμοί τους, ειδικά αυτοί που χρησιμοποιούνται για εποπτεία περιεχομένου, σύσταση και κατάταξη. Οι πλατφόρμες πρέπει να εξηγούν τα κριτήρια και τη λογική πίσω από τους αλγόριθμούς τους, επιτρέποντας στους χρήστες να κατανοούν πώς λαμβάνονται οι αποφάσεις και πώς παρουσιάζεται το περιεχόμενο σε αυτούς. Οι VLOPs έχουν πρόσθετες υποχρεώσεις να διεξάγουν ελέγχους αλγορίθμων και να επιτρέπουν σε ανεξάρτητους ερευνητές να αξιολογούν την επίδραση των αλγορίθμων τους στην κοινωνία. Aυτά τα μέτρα διαφάνειας αποσκοπούν στην αύξηση της λογοδοσίας και της εμπιστοσύνης στο ψηφιακό οικοσύστημα."
},
{
    'law': 'gdpr',
    'question': "Ποιος είναι ο ρόλος του Υπευθύνου Προστασίας Δεδομένων (DPO) σύμφωνα με τον GDPR;",
    'answer': "Ο Υπεύθυνος Προστασίας Δεδομένων (DPO) είναι υπεύθυνος για την εποπτεία των στρατηγικών προστασίας δεδομένων και την εξασφάλιση της συμμόρφωσης με τις απαιτήσεις του GDPR. Ο DPO πρέπει να διοριστεί από τις δημόσιες αρχές και φορείς, και από οργανισμούς που ασχολούνται με τη συστηματική παρακολούθηση των υποκειμένων των δεδομένων σε μεγάλη κλίμακα ή επεξεργάζονται ειδικές κατηγορίες δεδομένων σε μεγάλη κλίμακα. Οι αρμοδιότητες του DPO περιλαμβάνουν τη συμβουλευτική προς τον οργανισμό σχετικά με τις υποχρεώσεις του GDPR, την παρακολούθηση της συμμόρφωσης, την εκπαίδευση του προσωπικού, τη διενέργεια ελέγχων και την εξυπηρέτηση ως σημείο επαφής για τις εποπτικές αρχές και τα υποκείμενα των δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιες είναι οι απαιτήσεις για τις αξιολογήσεις συμμόρφωσης βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου πρέπει να υποβάλλονται σε αξιολόγηση συμμόρφωσης πριν μπορέσουν να τοποθετηθούν στην αγορά ή να τεθούν σε λειτουργία. Αυτή η αξιολόγηση περιλαμβάνει την αξιολόγηση του κατά πόσον το σύστημα τεχνητής νοημοσύνης πληροί τις απαιτήσεις που ορίζονται στον Νόμο για την Τεχνητή Νοημοσύνη, όπως η διαχείριση κινδύνων, η διακυβέρνηση δεδομένων, η διαφάνεια, η ανθρώπινη επίβλεψη και η ακρίβεια. Η αξιολόγηση μπορεί να πραγματοποιηθεί από τον πάροχο ή από κοινοποιημένο οργανισμό, ανάλογα με τη φύση του συστήματος τεχνητής νοημοσύνης. Η αξιολόγηση συμμόρφωσης πρέπει να τεκμηριώνεται και το σύστημα τεχνητής νοημοσύνης πρέπει να φέρει σήμανση CE που να υποδεικνύει τη συμμόρφωση με τον κανονισμό."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα της πρακτικής σύζευξης και συνδυασμού από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαγορεύει στους διαμεσολαβητές να εμπλέκονται σε πρακτικές σύζευξης και συνδυασμού που απαιτούν από τους χρήστες να αγοράσουν ή να χρησιμοποιήσουν πρόσθετες υπηρεσίες ως προϋπόθεση για την πρόσβαση στην βασική υπηρεσία πλατφόρμας του διαμεσολαβητή. Για παράδειγμα, ένας διαμεσολαβητής δεν μπορεί να απαιτήσει από τους χρήστες να εγκαταστήσουν ή να χρησιμοποιήσουν μια συγκεκριμένη εφαρμογή ή υπηρεσία ως προϋπόθεση για τη χρήση της πλατφόρμας τους. Η απαγόρευση της σύζευξης και του συνδυασμού στοχεύει στο να αποτρέψει τους διαμεσολαβητές από το να αξιοποιούν τη δύναμή τους στην αγορά για να επεκτείνουν την κυριαρχία τους σε άλλες αγορές και να διασφαλίσουν ότι οι χρήστες έχουν την ελευθερία να επιλέγουν τις υπηρεσίες που θέλουν να χρησιμοποιήσουν."
},
{
    'law': 'dsa',
    'question': "Πώς αντιμετωπίζει ο DSA το ζήτημα της παραπληροφόρησης και των ψευδών ειδήσεων στις ψηφιακές πλατφόρμες;",
    'answer': "Ο DSA απαιτεί από τις πλατφόρμες, ειδικά τις VLOPs, να λαμβάνουν προληπτικά μέτρα για την καταπολέμηση της διάδοσης της παραπληροφόρησης και των ψευδών ειδήσεων. Αυτό περιλαμβάνει την εφαρμογή μηχανισμών για τον εντοπισμό, την αξιολόγηση και τη μείωση των κινδύνων που σχετίζονται με την παραπληροφόρηση, τη συνεργασία με ανεξάρτητους ελεγκτές γεγονότων και την παροχή στους χρήστες ακριβών πληροφοριών και πλαισίου. Οι πλατφόρμες πρέπει επίσης να διασφαλίζουν ότι τα συστήματα εποπτείας περιεχομένου και σύστασης τους δεν ενισχύουν ή προωθούν την παραπληροφόρηση. Ο DSA προάγει τη διαφάνεια απαιτώντας από τις πλατφόρμες να αναφέρουν τις προσπάθειές τους για την καταπολέμηση της παραπληροφόρησης και να παρέχουν στους χρήστες εργαλεία για τον εντοπισμό και την αναφορά ψευδών πληροφοριών."
},
{
    'law': 'gdpr',
    'question': "Ποιες είναι οι επιπτώσεις του GDPR για δραστηριότητες επεξεργασίας δεδομένων που εκτελούνται σε διασυνοριακό επίπεδο;",
    'answer': "Ο GDPR θεσπίζει ένα πλαίσιο για τις δραστηριότητες διασυνοριακής επεξεργασίας δεδομένων ώστε να διασφαλίζεται ότι η προστασία των δεδομένων είναι συνεπής σε όλη την ΕΕ. Οι οργανισμοί που επεξεργάζονται προσωπικά δεδομένα σε πολλά κράτη μέλη της ΕΕ πρέπει να ορίσουν μια επικεφαλής εποπτική αρχή, η οποία λειτουργεί ως το ενιαίο σημείο επαφής για την επίβλεψη της συμμόρφωσης. Ο GDPR διευκολύνει επίσης τη συνεργασία μεταξύ των εποπτικών αρχών μέσω μηχανισμών όπως ο μηχανισμός συνέπειας και το Ευρωπαϊκό Συμβούλιο Προστασίας Δεδομένων (EDPB)."
},
{
    'law': 'ai_act',
    'question': "Ποιο ρόλο διαδραματίζουν οι εθνικές εποπτικές αρχές βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Οι εθνικές εποπτικές αρχές είναι υπεύθυνες για την εποπτεία της εφαρμογής και της επιβολής του Νόμου για την Τεχνητή Νοημοσύνη στις αντίστοιχες δικαιοδοσίες τους. Αναλαμβάνουν την παρακολούθηση της συμμόρφωσης των συστημάτων τεχνητής νοημοσύνης με τις απαιτήσεις του νόμου, τη διεξαγωγή επιθεωρήσεων και ερευνών και τη λήψη μέτρων επιβολής όταν χρειάζεται. Αυτές οι αρχές διαδραματίζουν επίσης βασικό ρόλο στον συντονισμό με άλλες εθνικές αρχές και την Ευρωπαϊκή Επιτροπή για να διασφαλιστεί μια εναρμονισμένη προσέγγιση στη ρύθμιση της τεχνητής νοημοσύνης σε όλη την ΕΕ."
},
{
    'law': 'dma',
    'question': "Ποιες είναι οι συνέπειες για τους διαμεσολαβητές που αποτυγχάνουν να συμμορφωθούν με τον DMA;",
    'answer': "Οι διαμεσολαβητές που αποτυγχάνουν να συμμορφωθούν με τις υποχρεώσεις και τις απαγορεύσεις που ορίζονται στον DMA αντιμετωπίζουν σημαντικές συνέπειες, όπως πρόστιμα έως και 10% του συνολικού παγκόσμιου ετήσιου κύκλου εργασιών τους. Σε περιπτώσεις επανειλημμένης μη συμμόρφωσης, η Ευρωπαϊκή Επιτροπή μπορεί να επιβάλει επιπλέον μέτρα, όπως διαρθρωτικά μέτρα, συμπεριλαμβανομένης της διάθεσης μέρους των επιχειρήσεων. Ο DMA προβλέπει επίσης περιοδικές κυρώσεις πληρωμών για να διασφαλίσει ότι οι διαμεσολαβητές συμμορφώνονται με τις υποχρεώσεις σε συνεχή βάση. Η επιβολή του DMA έχει σχεδιαστεί να είναι ισχυρή για να αποτρέψει τους διαμεσολαβητές από την αντι-ανταγωνιστική συμπεριφορά."
},
{
    'law': 'dsa',
    'question': "Τι ρόλο παίζουν οι αξιόπιστοι ειδοποιητές (trusted flaggers) βάσει του DSA;",
    'answer': "Ο DSA αναγνωρίζει το ρόλο των αξιόπιστων ειδοποιητών—οντοτήτων με εξειδίκευση στον εντοπισμό παράνομου περιεχομένου—ως σημαντικούς συνεργάτες στην εποπτεία περιεχομένου. Οι αξιόπιστοι ειδοποιητές έχουν προτεραιότητα στους μηχανισμούς ειδοποίησης και δράσης, πράγμα που σημαίνει ότι οι αναφορές τους επεξεργάζονται πιο γρήγορα και με μεγαλύτερη ακρίβεια. Οι πλατφόρμες πρέπει να διασφαλίζουν ότι οι αναφορές των αξιόπιστων ειδοποιητών αντιμετωπίζονται από έμπειρους επόπτες και ότι λαμβάνουν ανατροφοδότηση για τις ενέργειες που λαμβάνονται. Ο ορισμός των αξιόπιστων ειδοποιητών έχει ως στόχο τη βελτίωση της αποδοτικότητας και της αποτελεσματικότητας της εποπτείας περιεχομένου, ιδιαίτερα στην καταπολέμηση του παράνομου περιεχομένου και των επιβλαβών δραστηριοτήτων στο διαδίκτυο."
},
{
    'law': 'gdpr',
    'question': "Πώς αντιμετωπίζει ο GDPR τις παραβιάσεις δεδομένων και ποιες είναι οι υποχρεώσεις των υπευθύνων επεξεργασίας σε τέτοιες περιπτώσεις;",
    'answer': "Σύμφωνα με τον GDPR, οι υπεύθυνοι επεξεργασίας δεδομένων απαιτείται να αναφέρουν παραβιάσεις δεδομένων στην αρμόδια εποπτική αρχή εντός 72 ωρών από τη στιγμή που γίνονται ενήμεροι για την παραβίαση, εκτός εάν η παραβίαση είναι απίθανο να προκαλέσει κίνδυνο στα δικαιώματα και τις ελευθερίες των ατόμων. Εάν η παραβίαση ενέχει υψηλό κίνδυνο για τα θιγόμενα άτομα, ο υπεύθυνος επεξεργασίας δεδομένων πρέπει επίσης να ενημερώσει τα υποκείμενα των δεδομένων χωρίς αδικαιολόγητη καθυστέρηση. Ο GDPR απαιτεί από τους οργανισμούς να εφαρμόζουν κατάλληλα τεχνικά και οργανωτικά μέτρα για την αποτροπή παραβιάσεων δεδομένων και τον μετριασμό των επιπτώσεών τους."
},
{
    'law': 'ai_act',
    'question': "Πώς ενθαρρύνει ο Νόμος για την Τεχνητή Νοημοσύνη την καινοτομία, διασφαλίζοντας παράλληλα την ασφάλεια και τη συμμόρφωση;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη ενθαρρύνει την καινοτομία παρέχοντας ρυθμιστικά sandbox, που είναι ελεγχόμενα περιβάλλοντα όπου οι προγραμματιστές τεχνητής νοημοσύνης μπορούν να δοκιμάσουν τα συστήματά τους υπό την επίβλεψη των αρμόδιων αρχών χωρίς να αντιμετωπίσουν αμέσως όλες τις κανονιστικές απαιτήσεις. Αυτά τα sandbox επιτρέπουν την πειραματική ανάπτυξη καινοτόμων λύσεων τεχνητής νοημοσύνης, διασφαλίζοντας παράλληλα ότι τηρούνται τα πρότυπα ασφαλείας, ηθικής και νομικής φύσης. Ο νόμος προωθεί επίσης την υιοθέτηση εθελοντικών κωδίκων δεοντολογίας για συστήματα τεχνητής νοημοσύνης που δεν ενέχουν υψηλό κίνδυνο, επιτρέποντας στους παρόχους να δείξουν τη δέσμευσή τους σε ηθικές πρακτικές τεχνητής νοημοσύνης."
},
{
    'law': 'dma',
    'question': "Πώς βελτιώνει ο DMA την προστασία των καταναλωτών στις ψηφιακές αγορές;",
    'answer': "Ο DMA βελτιώνει την προστασία των καταναλωτών διασφαλίζοντας ότι οι διαμεσολαβητές δεν εμπλέκονται σε πρακτικές που βλάπτουν τους καταναλωτές, όπως η αυτοπροτίμηση, οι άδικοι όροι και προϋποθέσεις ή ο περιορισμός της πρόσβασης στα δεδομένα. Ο DMA προάγει επίσης τη διαφάνεια στον τρόπο λειτουργίας των διαμεσολαβητών, απαιτώντας από αυτούς να παρέχουν σαφείς και προσβάσιμες πληροφορίες στους καταναλωτές για τις πρακτικές τους. Επιπλέον, ο DMA διασφαλίζει ότι οι καταναλωτές έχουν περισσότερη επιλογή και έλεγχο στις ψηφιακές υπηρεσίες που χρησιμοποιούν, προωθώντας τη διαλειτουργικότητα και τη φορητότητα δεδομένων. Με την προώθηση του ανταγωνισμού, ο DMA στοχεύει στη βελτίωση της ποιότητας και της οικονομικής προσβασιμότητας των ψηφιακών υπηρεσιών για τους καταναλωτές."
},
{
    'law': 'dsa',
    'question': "Πώς προάγει ο DSA τη λογοδοσία των διαδικτυακών πλατφορμών;",
    'answer': "Ο DSA προάγει τη λογοδοσία επιβάλλοντας αυστηρές απαιτήσεις αναφοράς και διαφάνειας στις διαδικτυακές πλατφόρμες. Οι πλατφόρμες πρέπει να δημοσιεύουν τακτικές εκθέσεις διαφάνειας που αναλύουν τις δραστηριότητες εποπτείας περιεχομένου τους, περιλαμβάνοντας τον αριθμό των ενεργειών αφαίρεσης, τους λόγους για τις αφαιρέσεις και τα αποτελέσματα των ενστάσεων των χρηστών. Οι VLOPs υποχρεούνται επίσης να υποβάλλονται σε ανεξάρτητους ελέγχους των πρακτικών εποπτείας περιεχομένου και διαχείρισης κινδύνων. Αυτοί οι έλεγχοι αποσκοπούν στην αξιολόγηση της συμμόρφωσης της πλατφόρμας με τον DSA και στον εντοπισμό περιοχών προς βελτίωση. Με την προώθηση της διαφάνειας και της λογοδοσίας, ο DSA στοχεύει στην οικοδόμηση εμπιστοσύνης στο ψηφιακό περιβάλλον και στη διασφάλιση ότι οι πλατφόρμες ενεργούν υπεύθυνα."
},
{
    'law': 'gdpr',
    'question': "Ποιοι είναι οι περιορισμοί στην επεξεργασία ειδικών κατηγοριών προσωπικών δεδομένων σύμφωνα με τον GDPR;",
    'answer': "Ο GDPR επιβάλλει αυστηρότερους κανόνες για την επεξεργασία ειδικών κατηγοριών προσωπικών δεδομένων, όπως δεδομένα που αποκαλύπτουν τη φυλετική ή εθνοτική καταγωγή, πολιτικές απόψεις, θρησκευτικές ή φιλοσοφικές πεποιθήσεις, συνδικαλιστική ιδιότητα, γενετικά δεδομένα, βιομετρικά δεδομένα, δεδομένα υγείας και δεδομένα που αφορούν τη σεξουαλική ζωή ή τον σεξουαλικό προσανατολισμό ενός ατόμου. Η επεξεργασία τέτοιων δεδομένων απαγορεύεται, εκτός εάν πληρούνται συγκεκριμένες προϋποθέσεις, όπως η ρητή συγκατάθεση του υποκειμένου των δεδομένων, η συμμόρφωση με νομικές υποχρεώσεις στον τομέα της απασχόλησης και της κοινωνικής ασφάλισης ή η προστασία των ζωτικών συμφερόντων του υποκειμένου των δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιες κατηγορίες συστημάτων τεχνητής νοημοσύνης θεωρούνται υψηλού κινδύνου βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου βάσει του Νόμου για την Τεχνητή Νοημοσύνη περιλαμβάνουν αυτά που χρησιμοποιούνται σε κρίσιμες υποδομές (όπως μεταφορές, ενέργεια και ύδρευση), εκπαίδευση και επαγγελματική κατάρτιση, διαχείριση της απασχόλησης και των εργαζομένων, πρόσβαση σε βασικές ιδιωτικές και δημόσιες υπηρεσίες (όπως πιστωτική αξιολόγηση και κοινωνικά επιδόματα), επιβολή του νόμου (όπως προληπτική αστυνόμευση), διαχείριση της μετανάστευσης, του ασύλου και των συνόρων, και τη διοίκηση της δικαιοσύνης και των δημοκρατικών διαδικασιών. Αυτά τα συστήματα υπόκεινται σε αυστηρές απαιτήσεις λόγω των σημαντικών κινδύνων που θέτουν στα θεμελιώδη δικαιώματα και την ασφάλεια."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα της πρόσβασης στα δεδομένα των επιχειρηματικών χρηστών από τους διαμεσολαβητές;",
    'answer': "Ο DMA επιβάλλει υποχρεώσεις στους διαμεσολαβητές να παρέχουν στους επιχειρηματικούς χρήστες πρόσβαση στα δεδομένα που δημιουργούνται μέσω των αλληλεπιδράσεών τους στην πλατφόρμα. Αυτό περιλαμβάνει πρόσβαση σε συγκεντρωτικά και ανώνυμα δεδομένα, καθώς και σε δεδομένα που είναι απαραίτητα για την ανάπτυξη και βελτίωση των προϊόντων και υπηρεσιών του επιχειρηματικού χρήστη. Ο DMA επίσης απαγορεύει στους διαμεσολαβητές να χρησιμοποιούν μη δημόσια δεδομένα από επιχειρηματικούς χρήστες για να ανταγωνίζονται εναντίον τους, διασφαλίζοντας ότι οι διαμεσολαβητές δεν εκμεταλλεύονται την πρόσβασή τους στα δεδομένα για να αποκτήσουν αθέμιτο ανταγωνιστικό πλεονέκτημα."
},
{
    'law': 'dsa',
    'question': "Ποιες είναι οι κυρώσεις για τη μη συμμόρφωση με τον DSA;",
    'answer': "Ο DSA προβλέπει σημαντικές κυρώσεις για τη μη συμμόρφωση, περιλαμβάνοντας πρόστιμα έως και 6% του συνολικού παγκόσμιου ετήσιου κύκλου εργασιών της πλατφόρμας. Σε περιπτώσεις επαναλαμβανόμενης ή σοβαρής μη συμμόρφωσης, ο DSA επιτρέπει πρόσθετα μέτρα, όπως η προσωρινή αναστολή των υπηρεσιών της πλατφόρμας ή άλλες διορθωτικές ενέργειες. Η επιβολή του DSA επιβλέπεται από τις εθνικές ρυθμιστικές αρχές, οι οποίες έχουν την εξουσία να διερευνούν και να επιβάλλουν κυρώσεις σε πλατφόρμες που παραβιάζουν τον κανονισμό. Αυτές οι κυρώσεις έχουν σχεδιαστεί για να διασφαλίσουν ότι οι πλατφόρμες λαμβάνουν σοβαρά υπόψη τις υποχρεώσεις τους και ότι οι διατάξεις του DSA εφαρμόζονται αποτελεσματικά."
},
{
    'law': 'gdpr',
    'question': "Πώς ρυθμίζει ο GDPR τη λήψη αυτοματοποιημένων αποφάσεων και την κατάρτιση προφίλ;",
    'answer': "Ο GDPR θέτει περιορισμούς στη λήψη αυτοματοποιημένων αποφάσεων, συμπεριλαμβανομένης της κατάρτισης προφίλ, όπου οι αποφάσεις λαμβάνονται αποκλειστικά βάσει αυτοματοποιημένης επεξεργασίας και επηρεάζουν σημαντικά τα άτομα. Τέτοια επεξεργασία επιτρέπεται μόνο σε συγκεκριμένες περιπτώσεις, όπως όταν είναι απαραίτητη για τη σύναψη ή εκτέλεση σύμβασης, όταν εξουσιοδοτείται από το δίκαιο της Ένωσης ή κράτους μέλους ή όταν βασίζεται στη ρητή συγκατάθεση του υποκειμένου των δεδομένων. Οι οργανισμοί πρέπει να διασφαλίζουν ότι τα άτομα ενημερώνονται για την ύπαρξη αυτοματοποιημένης λήψης αποφάσεων, τη λογική που χρησιμοποιείται και τις πιθανές συνέπειες. Τα υποκείμενα των δεδομένων έχουν το δικαίωμα να αντιταχθούν σε αυτοματοποιημένες αποφάσεις και να ζητήσουν ανθρώπινη παρέμβαση."
},
{
    'law': 'ai_act',
    'question': "Πώς αντιμετωπίζει ο Νόμος για την Τεχνητή Νοημοσύνη τη διαφάνεια των συστημάτων τεχνητής νοημοσύνης;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη απαιτεί τα συστήματα τεχνητής νοημοσύνης, ιδιαίτερα τα υψηλού κινδύνου, να σχεδιάζονται και να αναπτύσσονται με διαφάνεια. Αυτό περιλαμβάνει την παροχή σαφών και προσβάσιμων πληροφοριών στους χρήστες σχετικά με τον σκοπό, τις δυνατότητες, τους περιορισμούς και τον τρόπο λειτουργίας του συστήματος τεχνητής νοημοσύνης. Οι χρήστες πρέπει να ενημερώνονται όταν αλληλεπιδρούν με ένα σύστημα τεχνητής νοημοσύνης, ιδίως σε περιπτώσεις όπου η τεχνητή νοημοσύνη χρησιμοποιείται για να λαμβάνονται αποφάσεις που έχουν σημαντικές επιπτώσεις στα άτομα. Οι απαιτήσεις διαφάνειας στοχεύουν να διασφαλίσουν ότι οι χρήστες και τα άτομα που επηρεάζονται κατανοούν πώς και γιατί λαμβάνονται οι αποφάσεις από τα συστήματα τεχνητής νοημοσύνης."
},
{
    'law': 'dma',
    'question': "Πώς διασφαλίζει ο DMA την δίκαιη και μη διακριτική πρόσβαση στις βασικές υπηρεσίες πλατφόρμας;",
    'answer': "Ο DMA απαιτεί από τους διαμεσολαβητές να διασφαλίζουν ότι οι βασικές υπηρεσίες πλατφόρμας τους προσφέρονται με δίκαιους, λογικούς και μη διακριτικούς όρους. Αυτό σημαίνει ότι οι διαμεσολαβητές δεν μπορούν να επιβάλλουν άδικους όρους ή προϋποθέσεις στους επιχειρηματικούς χρήστες ή να εμπλέκονται σε πρακτικές που ευνοούν τις δικές τους υπηρεσίες έναντι αυτών των ανταγωνιστών. Ο DMA απαιτεί επίσης από τους διαμεσολαβητές να παρέχουν διαφάνεια στον τρόπο λειτουργίας τους, περιλαμβάνοντας σαφείς και προσβάσιμες πληροφορίες σχετικά με τους όρους και τις προϋποθέσεις για τη χρήση των υπηρεσιών τους. Αυτά τα μέτρα αποσκοπούν στο να αποτρέψουν τους διαμεσολαβητές από το να καταχρώνται την δύναμή τους στην αγορά και να διασφαλίσουν ένα ίσο επίπεδο ανταγωνισμού στις ψηφιακές αγορές."
},
{
    'law': 'dsa',
    'question': "Πώς αντιμετωπίζει ο DSA το ζήτημα της παρενόχλησης και της κακοποίησης στο διαδίκτυο;",
    'answer': "Ο DSA απαιτεί από τις πλατφόρμες να εφαρμόζουν μέτρα για την καταπολέμηση της διαδικτυακής παρενόχλησης και κακοποίησης, περιλαμβάνοντας την παροχή στους χρήστες εργαλείων για την αναφορά και τον αποκλεισμό καταχρηστικού περιεχομένου και συμπεριφοράς. Οι πλατφόρμες πρέπει να ενεργούν γρήγορα για την αφαίρεση ή την απενεργοποίηση της πρόσβασης σε περιεχόμενο που συνιστά παρενόχληση ή κακοποίηση και πρέπει να παρέχουν υποστήριξη στα θύματα. Ο DSA ενθαρρύνει επίσης τις πλατφόρμες να συνεργάζονται με τις αρχές επιβολής του νόμου και οργανώσεις της κοινωνίας των πολιτών για την αντιμετώπιση της διαδικτυακής παρενόχλησης και να αναπτύξουν βέλτιστες πρακτικές για τη δημιουργία ενός ασφαλούς διαδικτυακού περιβάλλοντος. Αυτά τα μέτρα αποσκοπούν στην προστασία των χρηστών από βλάβες και στην προώθηση ενός σεβαστικού και χωρίς αποκλεισμούς ψηφιακού χώρου."
},
{
    'law': 'gdpr',
    'question': "Ποιες κυρώσεις και ενέργειες επιβολής προβλέπει ο GDPR;",
    'answer': "Ο GDPR προβλέπει σημαντικές κυρώσεις και ενέργειες επιβολής για τη διασφάλιση της συμμόρφωσης. Οι εποπτικές αρχές έχουν την εξουσία να επιβάλλουν διοικητικά πρόστιμα έως 20 εκατομμύρια ευρώ ή 4% του συνολικού παγκόσμιου ετήσιου κύκλου εργασιών του προηγούμενου οικονομικού έτους, όποιο είναι μεγαλύτερο, για τις πιο σοβαρές παραβάσεις. Οι κυρώσεις καθορίζονται με βάση παράγοντες όπως η φύση, η σοβαρότητα και η διάρκεια της παράβασης, ο σκόπιμος ή αμελής χαρακτήρας της παράβασης και τα μέτρα που έλαβε ο οργανισμός για τον μετριασμό των ζημιών."
},
{
    'law': 'ai_act',
    'question': "Ποιες είναι οι απαιτήσεις για τις αξιολογήσεις συμμόρφωσης βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Τα συστήματα τεχνητής νοημοσύνης υψηλού κινδύνου πρέπει να υποβάλλονται σε αξιολόγηση συμμόρφωσης πριν μπορέσουν να τοποθετηθούν στην αγορά ή να τεθούν σε λειτουργία. Αυτή η αξιολόγηση περιλαμβάνει την αξιολόγηση του κατά πόσον το σύστημα τεχνητής νοημοσύνης πληροί τις απαιτήσεις που ορίζονται στον Νόμο για την Τεχνητή Νοημοσύνη, όπως η διαχείριση κινδύνων, η διακυβέρνηση δεδομένων, η διαφάνεια, η ανθρώπινη επίβλεψη και η ακρίβεια. Η αξιολόγηση μπορεί να πραγματοποιηθεί από τον πάροχο ή από κοινοποιημένο οργανισμό, ανάλογα με τη φύση του συστήματος τεχνητής νοημοσύνης. Η αξιολόγηση συμμόρφωσης πρέπει να τεκμηριώνεται και το σύστημα τεχνητής νοημοσύνης πρέπει να φέρει σήμανση CE που να υποδεικνύει τη συμμόρφωση με τον κανονισμό."
},
{
    'law': 'dma',
    'question': "Πώς προάγει ο DMA την καινοτομία και τον ανταγωνισμό στις ψηφιακές αγορές;",
    'answer': "Ο DMA προάγει την καινοτομία και τον ανταγωνισμό αποτρέποντας τους διαμεσολαβητές από το να εμπλέκονται σε πρακτικές που καταστέλλουν τον ανταγωνισμό, όπως η αυτοπροτίμηση, η σύζευξη και ο συνδυασμός. Εξασφαλίζοντας ότι οι διαμεσολαβητές λειτουργούν με δίκαιους, λογικούς και μη διακριτικούς όρους, ο DMA δημιουργεί ευκαιρίες για νέες εισόδους και μικρότερους ανταγωνιστές να ανταγωνίζονται σε ίσο επίπεδο. Ο DMA προάγει επίσης τη διαλειτουργικότητα και τη φορητότητα δεδομένων, επιτρέποντας στις επιχειρήσεις να αναπτύξουν καινοτόμες υπηρεσίες που μπορούν να αλληλεπιδρούν με την πλατφόρμα του διαμεσολαβητή. Αυτά τα μέτρα έχουν σχεδιαστεί για να ενισχύσουν μια δυναμική και ανταγωνιστική ψηφιακή αγορά που ωφελεί τόσο τους καταναλωτές όσο και τις επιχειρήσεις."
},
{
    'law': 'dsa',
    'question': "Πώς διασφαλίζει ο DSA ότι οι χρήστες έχουν έλεγχο στα δεδομένα και το απόρρητό τους;",
    'answer': "Ο DSA ενισχύει τον έλεγχο των χρηστών στα δεδομένα και το απόρρητό τους απαιτώντας από τις πλατφόρμες να παρέχουν σαφείς και προσβάσιμες πληροφορίες για το πώς συλλέγονται, επεξεργάζονται και χρησιμοποιούνται τα δεδομένα των χρηστών. Οι χρήστες πρέπει να ενημερώνονται για τα δικαιώματά τους να έχουν πρόσβαση, να διορθώνουν και να διαγράφουν τα δεδομένα τους, καθώς και για το δικαίωμά τους να αντιτάσσονται στην επεξεργασία δεδομένων. Ο DSA απαιτεί επίσης από τις πλατφόρμες να εφαρμόζουν αρχές προστασίας της ιδιωτικότητας εξ αρχής και από προεπιλογή (privacy-by-design και privacy-by-default), διασφαλίζοντας ότι το απόρρητο των χρηστών προστατεύεται από την αρχή. Επιπλέον, οι πλατφόρμες πρέπει να παρέχουν στους χρήστες εργαλεία για τη διαχείριση των ρυθμίσεων απορρήτου τους και για τον έλεγχο της χρήσης των δεδομένων τους για στοχευμένη διαφήμιση."
},
{
    'law': 'gdpr',
    'question': "Ποιος είναι ο ρόλος του Ευρωπαϊκού Συμβουλίου Προστασίας Δεδομένων (EDPB) σύμφωνα με τον GDPR;",
    'answer': "Το Ευρωπαϊκό Συμβούλιο Προστασίας Δεδομένων (EDPB) είναι ένα ανεξάρτητο όργανο που ιδρύθηκε από τον GDPR για να εξασφαλίσει την ομοιόμορφη εφαρμογή των κανόνων προστασίας δεδομένων σε όλη την ΕΕ. Το EDPB απαρτίζεται από εκπροσώπους των εθνικών αρχών προστασίας δεδομένων και του Ευρωπαίου Επόπτη Προστασίας Δεδομένων (EDPS). Οι αρμοδιότητές του περιλαμβάνουν την έκδοση κατευθυντήριων γραμμών, συστάσεων και βέλτιστων πρακτικών σχετικά με την ερμηνεία και εφαρμογή του GDPR, την επίλυση διαφορών μεταξύ εποπτικών αρχών και την παροχή συμβουλών στην Ευρωπαϊκή Επιτροπή για θέματα προστασίας δεδομένων."
},
{
    'law': 'ai_act',
    'question': "Ποιο ρόλο διαδραματίζουν οι εθνικές εποπτικές αρχές βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Οι εθνικές εποπτικές αρχές είναι υπεύθυνες για την εποπτεία της εφαρμογής και της επιβολής του Νόμου για την Τεχνητή Νοημοσύνη στις αντίστοιχες δικαιοδοσίες τους. Αναλαμβάνουν την παρακολούθηση της συμμόρφωσης των συστημάτων τεχνητής νοημοσύνης με τις απαιτήσεις του νόμου, τη διεξαγωγή επιθεωρήσεων και ερευνών και τη λήψη μέτρων επιβολής όταν χρειάζεται. Αυτές οι αρχές διαδραματίζουν επίσης βασικό ρόλο στον συντονισμό με άλλες εθνικές αρχές και την Ευρωπαϊκή Επιτροπή για να διασφαλιστεί μια εναρμονισμένη προσέγγιση στη ρύθμιση της τεχνητής νοημοσύνης σε όλη την ΕΕ."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα των σκοτεινών σχεδίων και παραπλανητικών πρακτικών σχεδιασμού από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαγορεύει στους διαμεσολαβητές να χρησιμοποιούν σκοτεινά σχέδια και παραπλανητικές πρακτικές σχεδιασμού που χειραγωγούν ή εξαπατούν τους χρήστες να λαμβάνουν αποφάσεις που δεν είναι προς το συμφέρον τους. Αυτό περιλαμβάνει πρακτικές όπως η απόκρυψη σημαντικών πληροφοριών, η δυσκολία για τους χρήστες να ασκήσουν τα δικαιώματά τους, ή η ώθηση των χρηστών προς συγκεκριμένες επιλογές. Ο DMA απαιτεί από τους διαμεσολαβητές να παρέχουν σαφείς και προσβάσιμες πληροφορίες στους χρήστες και να σχεδιάζουν τις διεπαφές τους με τρόπο που να σέβεται την αυτονομία και την επιλογή των χρηστών. Αυτές οι διατάξεις αποσκοπούν στην προστασία των καταναλωτών από παραπλανητικές πρακτικές και στη διασφάλιση ότι οι ψηφιακές υπηρεσίες είναι διαφανείς και φιλικές προς τον χρήστη."
},
{
    'law': 'dsa',
    'question': "Ποιες είναι οι απαιτήσεις για τις διαδικτυακές πλατφόρμες να συνεργάζονται με τις ρυθμιστικές αρχές βάσει του DSA;",
    'answer': "Ο DSA απαιτεί από τις διαδικτυακές πλατφόρμες να συνεργάζονται με τις ρυθμιστικές αρχές παρέχοντάς τους πρόσβαση σε δεδομένα, αρχεία και πληροφορίες που είναι απαραίτητες για σκοπούς παρακολούθησης και επιβολής. Οι πλατφόρμες πρέπει να ανταποκρίνονται άμεσα στα αιτήματα των αρχών και να διευκολύνουν τις επιθεωρήσεις και τις έρευνες. Ο DSA επίσης επιβάλλει στις πλατφόρμες να παρέχουν εκθέσεις διαφάνειας και να υποβάλλονται σε ανεξάρτητους ελέγχους για να αποδεικνύουν τη συμμόρφωση με τον κανονισμό. Η συνεργασία με τις αρχές είναι απαραίτητη για τη διασφάλιση ότι οι πλατφόρμες πληρούν τις υποχρεώσεις τους και ότι οι διατάξεις του DSA εφαρμόζονται αποτελεσματικά."
},
{
    'law': 'gdpr',
    'question': "Πώς αντιμετωπίζει ο GDPR το ζήτημα της συγκατάθεσης στην επεξεργασία δεδομένων;",
    'answer': "Σύμφωνα με τον GDPR, η συγκατάθεση πρέπει να είναι ελεύθερη, συγκεκριμένη, ενημερωμένη και σαφής. Οι οργανισμοί πρέπει να διασφαλίζουν ότι η συγκατάθεση λαμβάνεται μέσω σαφούς θετικής ενέργειας, όπως το τσεκάρισμα ενός πλαισίου σε μια ιστοσελίδα, και ότι είναι διακριτή από άλλα θέματα. Το υποκείμενο των δεδομένων πρέπει να ενημερώνεται για το δικαίωμά του να αποσύρει τη συγκατάθεση του οποτεδήποτε και η ανάκληση πρέπει να είναι εξίσου εύκολη με τη χορήγηση της συγκατάθεσης. Επιπλέον, για τα παιδιά κάτω των 16 ετών, απαιτείται η συγκατάθεση του γονέα για την επεξεργασία των δεδομένων τους."
},
{
    'law': 'ai_act',
    'question': "Πώς ενθαρρύνει ο Νόμος για την Τεχνητή Νοημοσύνη την καινοτομία, διασφαλίζοντας παράλληλα την ασφάλεια και τη συμμόρφωση;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη ενθαρρύνει την καινοτομία παρέχοντας ρυθμιστικά sandbox, που είναι ελεγχόμενα περιβάλλοντα όπου οι προγραμματιστές τεχνητής νοημοσύνης μπορούν να δοκιμάσουν τα συστήματά τους υπό την επίβλεψη των αρμόδιων αρχών χωρίς να αντιμετωπίσουν αμέσως όλες τις κανονιστικές απαιτήσεις. Αυτά τα sandbox επιτρέπουν την πειραματική ανάπτυξη καινοτόμων λύσεων τεχνητής νοημοσύνης, διασφαλίζοντας παράλληλα ότι τηρούνται τα πρότυπα ασφαλείας, ηθικής και νομικής φύσης. Ο νόμος προωθεί επίσης την υιοθέτηση εθελοντικών κωδίκων δεοντολογίας για συστήματα τεχνητής νοημοσύνης που δεν ενέχουν υψηλό κίνδυνο, επιτρέποντας στους παρόχους να δείξουν τη δέσμευσή τους σε ηθικές πρακτικές τεχνητής νοημοσύνης."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα των συγχωνεύσεων και εξαγορών από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαιτεί από τους διαμεσολαβητές να ενημερώνουν την Ευρωπαϊκή Επιτροπή για οποιεσδήποτε προγραμματισμένες συγχωνεύσεις, εξαγορές ή συγκεντρώσεις που αφορούν άλλους παρόχους βασικών υπηρεσιών πλατφόρμας ή ψηφιακών υπηρεσιών. Αυτή η απαίτηση γνωστοποίησης επιτρέπει στην Επιτροπή να αξιολογήσει εάν η προτεινόμενη συναλλαγή θα υπονομεύσει τους στόχους του DMA, όπως ενισχύοντας τη δύναμη της αγοράς του διαμεσολαβητή ή μειώνοντας τον ανταγωνισμό στις ψηφιακές αγορές. Οι διατάξεις του DMA για τις συγχωνεύσεις και εξαγορές αποσκοπούν στην αποτροπή των διαμεσολαβητών από το να εδραιώνουν την κυριαρχία τους μέσω στρατηγικών εξαγορών και να διασφαλίσουν ότι ο ανταγωνισμός παραμένει ισχυρός στις ψηφιακές αγορές."
},
{
    'law': 'dsa',
    'question': "Πώς αντιμετωπίζει ο DSA το ζήτημα της διαφάνειας και της λογοδοσίας των αλγορίθμων;",
    'answer': "Ο DSA απαιτεί από τις πλατφόρμες, ιδιαίτερα τις VLOPs, να παρέχουν διαφάνεια σχετικά με το πώς λειτουργούν οι αλγόριθμοί τους, περιλαμβάνοντας τα κριτήρια που χρησιμοποιούνται για την πρόταση, την κατάταξη και την αφαίρεση περιεχομένου. Οι πλατφόρμες πρέπει να εξηγούν τη λογική πίσω από τους αλγόριθμους και να παρέχουν στους χρήστες επιλογές για να ελέγχουν πώς οι αλγόριθμοι επηρεάζουν την εμπειρία τους στο διαδίκτυο. Ο DSA επίσης επιβάλλει στις πλατφόρμες να διενεργούν τακτικούς ελέγχους των αλγορίθμων τους για να αξιολογούν τον αντίκτυπό τους στους χρήστες και την κοινωνία. Αυτοί οι έλεγχοι πρέπει να διενεργούνται από ανεξάρτητους τρίτους και πρέπει να αξιολογούν κατά πόσο οι αλγόριθμοι είναι δίκαιοι, μη-διακριτικοί και ευθυγραμμισμένοι με τα θεμελιώδη δικαιώματα."
},
{
    'law': 'gdpr',
    'question': "Ποια είναι η προσέγγιση του GDPR στις διεθνείς μεταφορές δεδομένων;",
    'answer': "Ο GDPR επιτρέπει τις διεθνείς μεταφορές δεδομένων μόνο εάν η τρίτη χώρα, η επικράτεια ή ο διεθνής οργανισμός εξασφαλίζει επαρκές επίπεδο προστασίας των δεδομένων, όπως καθορίζεται από την Ευρωπαϊκή Επιτροπή. Ελλείψει απόφασης επάρκειας, οι μεταφορές επιτρέπονται υπό κατάλληλες διασφαλίσεις, όπως δεσμευτικοί εταιρικοί κανόνες ή τυποποιημένες συμβατικές ρήτρες. Σε συγκεκριμένες περιπτώσεις, εξαιρέσεις για ειδικές καταστάσεις, όπως η ρητή συγκατάθεση του υποκειμένου των δεδομένων, μπορούν να επιτρέπουν μεταφορές. Ο GDPR στοχεύει να διασφαλίσει ότι τα προσωπικά δεδομένα που μεταφέρονται εκτός ΕΕ απολαμβάνουν το ίδιο επίπεδο προστασίας με αυτό εντός της ΕΕ."
},
{
    'law': 'ai_act',
    'question': "Πώς αντιμετωπίζει ο Νόμος για την Τεχνητή Νοημοσύνη το ζήτημα της προκατάληψης και της διάκρισης στα συστήματα τεχνητής νοημοσύνης;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη απαιτεί τα συστήματα τεχνητής νοημοσύνης, ιδιαίτερα τα υψηλού κινδύνου, να σχεδιάζονται και να αναπτύσσονται με τρόπο που να αποτρέπει, να εντοπίζει και να μετριάζει τις προκαταλήψεις που θα μπορούσαν να οδηγήσουν σε διακριτικά αποτελέσματα. Οι πάροχοι πρέπει να λαμβάνουν μέτρα για να διασφαλίζουν ότι τα συστήματα τεχνητής νοημοσύνης δεν παράγουν αποτελέσματα που αδικαιολόγητα μειονεκτούν άτομα ή ομάδες με βάση τα προστατευόμενα χαρακτηριστικά όπως η φυλή, το φύλο ή η θρησκεία. Αυτό περιλαμβάνει τη χρήση ποικίλων συνόλων δεδομένων, τη διενέργεια ελέγχων προκατάληψης και την εφαρμογή διορθωτικών μέτρων για την αντιμετώπιση τυχόν εντοπισμένων προκαταλήψεων. Ο νόμος τονίζει επίσης τη σημασία της ανθρώπινης επίβλεψης στην πρόληψη και αντιμετώπιση της προκατάληψης."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα των σκοτεινών σχεδίων και παραπλανητικών πρακτικών σχεδιασμού από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαγορεύει στους διαμεσολαβητές να χρησιμοποιούν σκοτεινά σχέδια και παραπλανητικές πρακτικές σχεδιασμού που χειραγωγούν ή εξαπατούν τους χρήστες να λαμβάνουν αποφάσεις που δεν είναι προς το συμφέρον τους. Αυτό περιλαμβάνει πρακτικές όπως η απόκρυψη σημαντικών πληροφοριών, η δυσκολία για τους χρήστες να ασκήσουν τα δικαιώματά τους, ή η ώθηση των χρηστών προς συγκεκριμένες επιλογές. Ο DMA απαιτεί από τους διαμεσολαβητές να παρέχουν σαφείς και προσβάσιμες πληροφορίες στους χρήστες και να σχεδιάζουν τις διεπαφές τους με τρόπο που να σέβεται την αυτονομία και την επιλογή των χρηστών. Αυτές οι διατάξεις αποσκοπούν στην προστασία των καταναλωτών από παραπλανητικές πρακτικές και στη διασφάλιση ότι οι ψηφιακές υπηρεσίες είναι διαφανείς και φιλικές προς τον χρήστη."
},
{
    'law': 'dsa',
    'question': "Πώς προάγει ο DSA την ανάπτυξη κωδίκων δεοντολογίας για τις διαδικτυακές πλατφόρμες;",
    'answer': "Ο DSA ενθαρρύνει την ανάπτυξη κωδίκων δεοντολογίας για τις διαδικτυακές πλατφόρμες για να αντιμετωπιστούν συγκεκριμένα ζητήματα όπως η εποπτεία περιεχομένου, η διαφάνεια των αλγορίθμων και η προστασία των ανηλίκων. Αυτοί οι κώδικες δεοντολογίας αναπτύσσονται σε συνεργασία με φορείς του κλάδου, οργανώσεις της κοινωνίας των πολιτών και ρυθμιστικές αρχές. Ο DSA προάγει την υιοθέτηση αυτών των εθελοντικών μέτρων για να διασφαλίσει ότι οι πλατφόρμες λειτουργούν με υπεύθυνο και ηθικό τρόπο. Οι κώδικες δεοντολογίας παρέχουν ένα πλαίσιο για βέλτιστες πρακτικές και βοηθούν τις πλατφόρμες να ευθυγραμμίσουν τις δραστηριότητές τους με τους στόχους του DSA, επιτρέποντας παράλληλα ευελιξία και καινοτομία."
},
{
    'law': 'gdpr',
    'question': "Πώς επηρεάζει ο GDPR τις μικρές και μεσαίες επιχειρήσεις (ΜΜΕ);",
    'answer': "Ο GDPR αναγνωρίζει τις προκλήσεις που αντιμετωπίζουν οι μικρές και μεσαίες επιχειρήσεις (ΜΜΕ) και παρέχει ορισμένες εξαιρέσεις και παρεκκλίσεις για την ελάφρυνση του φορτίου συμμόρφωσης. Για παράδειγμα, οι ΜΜΕ με λιγότερους από 250 εργαζόμενους δεν απαιτείται να διατηρούν αρχεία δραστηριοτήτων επεξεργασίας εκτός εάν η επεξεργασία είναι τακτική, ενέχει κίνδυνο για τα δικαιώματα και τις ελευθερίες των υποκειμένων των δεδομένων ή περιλαμβάνει ειδικές κατηγορίες δεδομένων. Ωστόσο, οι ΜΜΕ πρέπει να συμμορφώνονται με άλλες απαιτήσεις του GDPR, όπως η ειδοποίηση παραβίασης δεδομένων, τα δικαιώματα των υποκειμένων των δεδομένων και ο διορισμός ΥΠΔ, εάν είναι απαραίτητο."
},
{
    'law': 'ai_act',
    'question': "Ποιος είναι ο ρόλος του Ευρωπαϊκού Συμβουλίου Τεχνητής Νοημοσύνης (EAIB) βάσει του Νόμου για την Τεχνητή Νοημοσύνη;",
    'answer': "Το Ευρωπαϊκό Συμβούλιο Τεχνητής Νοημοσύνης (EAIB) ιδρύεται βάσει του Νόμου για την Τεχνητή Νοημοσύνη για να διευκολύνει τη συνεργασία και τον συντονισμό μεταξύ των εθνικών εποπτικών αρχών και της Ευρωπαϊκής Επιτροπής. Το EAIB είναι υπεύθυνο για την έκδοση κατευθυντήριων γραμμών, συστάσεων και βέλτιστων πρακτικών σχετικά με την εφαρμογή του Νόμου για την Τεχνητή Νοημοσύνη, την παροχή συμβουλών στην Ευρωπαϊκή Επιτροπή για θέματα που σχετίζονται με την τεχνητή νοημοσύνη και την προώθηση της εναρμονισμένης εφαρμογής του νόμου σε ολόκληρη την ΕΕ. Το EAIB διαδραματίζει επίσης ρόλο στην επίλυση διαφορών μεταξύ των εθνικών αρχών και στην εξασφάλιση συνέπειας στην ερμηνεία και την επιβολή του Νόμου για την Τεχνητή Νοημοσύνη."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα της αυτοπροτίμησης από τους διαμεσολαβητές;",
    'answer': "Ο DMA απαγορεύει ειδικά στους διαμεσολαβητές να εμπλέκονται σε πρακτικές αυτοπροτίμησης, όπου ευνοούν τα δικά τους προϊόντα ή υπηρεσίες έναντι αυτών των ανταγωνιστών στις πλατφόρμες τους. Αυτό περιλαμβάνει πρακτικές όπως η κατάταξη των δικών τους προϊόντων ψηλότερα στα αποτελέσματα αναζήτησης ή η παροχή προνομιακής πρόσβασης σε δεδομένα. Ο στόχος είναι να διασφαλιστεί ένα ίσο επίπεδο ανταγωνισμού στις ψηφιακές αγορές, όπου ο ανταγωνισμός βασίζεται στην αξία και όχι στη δύναμη της αγοράς του διαμεσολαβητή. Η απαγόρευση της αυτοπροτίμησης είναι μία από τις βασικές υποχρεώσεις που επιβάλλονται στους διαμεσολαβητές για να αποτραπεί η αντι-ανταγωνιστική συμπεριφορά."
},
{
    'law': 'dsa',
    'question': "Πώς αντιμετωπίζει ο DSA το ζήτημα της παραπληροφόρησης και των ψευδών ειδήσεων στις ψηφιακές πλατφόρμες;",
    'answer': "Ο DSA απαιτεί από τις πλατφόρμες, ειδικά τις VLOPs, να λαμβάνουν προληπτικά μέτρα για την καταπολέμηση της διάδοσης της παραπληροφόρησης και των ψευδών ειδήσεων. Αυτό περιλαμβάνει την εφαρμογή μηχανισμών για τον εντοπισμό, την αξιολόγηση και τη μείωση των κινδύνων που σχετίζονται με την παραπληροφόρηση, τη συνεργασία με ανεξάρτητους ελεγκτές γεγονότων και την παροχή στους χρήστες ακριβών πληροφοριών και πλαισίου. Οι πλατφόρμες πρέπει επίσης να διασφαλίζουν ότι τα συστήματα εποπτείας περιεχομένου και σύστασης τους δεν ενισχύουν ή προωθούν την παραπληροφόρηση. Ο DSA προάγει τη διαφάνεια απαιτώντας από τις πλατφόρμες να αναφέρουν τις προσπάθειές τους για την καταπολέμηση της παραπληροφόρησης και να παρέχουν στους χρήστες εργαλεία για τον εντοπισμό και την αναφορά ψευδών πληροφοριών."
},
{
    'law': 'gdpr',
    'question': "Ποια μέτρα συνιστά ο GDPR για τη διασφάλιση της ασφάλειας των δεδομένων;",
    'answer': "Ο GDPR απαιτεί από τους οργανισμούς να εφαρμόζουν κατάλληλα τεχνικά και οργανωτικά μέτρα για να εξασφαλίζουν επίπεδο ασφάλειας κατάλληλο για τον κίνδυνο. Αυτό περιλαμβάνει μέτρα όπως η ψευδωνυμοποίηση και η κρυπτογράφηση προσωπικών δεδομένων, η διασφάλιση της συνεχούς εμπιστευτικότητας, ακεραιότητας, διαθεσιμότητας και ανθεκτικότητας των συστημάτων και υπηρεσιών επεξεργασίας και η τακτική δοκιμή, αξιολόγηση και αξιολόγηση της αποτελεσματικότητας των μέτρων ασφαλείας. Ο GDPR δίνει επίσης έμφαση στην ανάγκη για εκτιμήσεις επιπτώσεων στην προστασία δεδομένων (DPIAs) όταν οι εργασίες επεξεργασίας είναι πιθανό να οδηγήσουν σε υψηλό κίνδυνο για τα δικαιώματα και τις ελευθερίες των φυσικών προσώπων."
},
{
    'law': 'ai_act',
    'question': "Πώς επηρεάζει ο Νόμος για την Τεχνητή Νοημοσύνη τη χρήση της τεχνητής νοημοσύνης στην υγειονομική περίθαλψη;",
    'answer': "Ο Νόμος για την Τεχνητή Νοημοσύνη αναγνωρίζει τα πιθανά οφέλη της τεχνητής νοημοσύνης στην υγειονομική περίθαλψη, όπως η βελτίωση της διάγνωσης, της θεραπείας και των αποτελεσμάτων των ασθενών. Ωστόσο, αναγνωρίζει επίσης τους κινδύνους που συνδέονται με τη χρήση της τεχνητής νοημοσύνης σε αυτόν τον ευαίσθητο τομέα. Τα συστήματα τεχνητής νοημοσύνης που χρησιμοποιούνται στην υγειονομική περίθαλψη, ιδιαίτερα αυτά που περιλαμβάνουν τη λήψη αποφάσεων ή παρέχουν συστάσεις σε επαγγελματίες υγείας, κατατάσσονται ως υψηλού κινδύνου και υπόκεινται σε αυστηρές απαιτήσεις. Αυτές περιλαμβάνουν τη διασφάλιση της ακρίβειας και της αξιοπιστίας των συστημάτων τεχνητής νοημοσύνης, τη διατήρηση ανθρώπινης επίβλεψης και την προστασία των δεδομένων των ασθενών. Ο νόμος τονίζει επίσης τη σημασία της διαφάνειας και της ενημερωμένης συναίνεσης στη χρήση της τεχνητής νοημοσύνης στην υγειονομική περίθαλψη."
},
{
    'law': 'dma',
    'question': "Πώς αντιμετωπίζει ο DMA το ζήτημα της πρόσβασης στις βασικές υπηρεσίες πλατφόρμας από τους τελικούς χρήστες;",
    'answer': "Ο DMA διασφαλίζει ότι οι τελικοί χρήστες έχουν πρόσβαση στις βασικές υπηρεσίες πλατφόρμας με δίκαιους και μη διακριτικούς όρους. Οι διαμεσολαβητές απαγορεύεται να περιορίζουν ή να υποβαθμίζουν την ποιότητα πρόσβασης στις υπηρεσίες τους ή να εμπλέκονται σε πρακτικές που περιορίζουν την επιλογή των χρηστών, όπως να αναγκάζουν τους χρήστες να εγκαταστήσουν ορισμένες εφαρμογές ή να χρησιμοποιούν συγκεκριμένες υπηρεσίες. Ο DMA προάγει επίσης τη φορητότητα δεδομένων, επιτρέποντας στους τελικούς χρήστες να μεταφέρουν τα δεδομένα τους σε άλλες υπηρεσίες και να επωφελούνται από ανταγωνιστικές προσφορές. Αυτές οι διατάξεις έχουν σχεδιαστεί για να ενισχύσουν την επιλογή και τον έλεγχο των χρηστών στις ψηφιακές υπηρεσίες που χρησιμοποιούν."
},
{
    'law': 'dsa',
    'question': "Ποιες είναι οι κυρώσεις για τη μη συμμόρφωση με τον DSA;",
    'answer': "Ο DSA προβλέπει σημαντικές κυρώσεις για τη μη συμμόρφωση, περιλαμβάνοντας πρόστιμα έως και 6% του συνολικού παγκόσμιου ετήσιου κύκλου εργασιών της πλατφόρμας. Σε περιπτώσεις επαναλαμβανόμενης ή σοβαρής μη συμμόρφωσης, ο DSA επιτρέπει πρόσθετα μέτρα, όπως η προσωρινή αναστολή των υπηρεσιών της πλατφόρμας ή άλλες διορθωτικές ενέργειες. Η επιβολή του DSA επιβλέπεται από τις εθνικές ρυθμιστικές αρχές, οι οποίες έχουν την εξουσία να διερευνούν και να επιβάλλουν κυρώσεις σε πλατφόρμες που παραβιάζουν τον κανονισμό. Αυτές οι κυρώσεις έχουν σχεδιαστεί για να διασφαλίσουν ότι οι πλατφόρμες λαμβάνουν σοβαρά υπόψη τις υποχρεώσεις τους και ότι οι διατάξεις του DSA εφαρμόζονται αποτελεσματικά."
}
]

laws_info = {
    'gdpr': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_gdpr.html',
        'collection_name': 'chunk-embeddings-index',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'gdpr']
    },
    'ai_act': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_AI_act.html',
        'collection_name': 'chunk-embeddings-index',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'ai_act']
    },
    'dma': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_dma.html',
        'collection_name': 'DMA_Chunk',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dma']
    },
    'dsa': {
        'file_path': '/kaggle/input/greek-laws-and-qa/greek_dsa.html',
        'collection_name': 'DSA_Chunk',
        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dsa']
    },
}

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  
print(f"Summarizer device: {summarizer.device}")
print(f"Cosine model device: {cosine_model.device}")
print(f"Semantic model device: {semantic_model.device}")
model.device

In [ ]:
from collections import defaultdict
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
summarizer = pipeline("summarization", model="IMISLab/GreekT5-mt5-small-greeksum",device=0 if torch.cuda.is_available() else -1)
cosine_model = SentenceTransformer('nlpaueb/bert-base-greek-uncased-v1',device=device) 


def embed_query(query, model_name):
    return cosine_model.encode(query, convert_to_tensor=True)
    
def summarize_text_huggingface_with_retry(text, max_length=350, min_length=100, max_retries=3):
    for attempt in range(max_retries):
        try:
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print("Max retries reached. Returning None.")
                return None

def calculate_cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    
for law, law_info in laws_info.items():
    print(f"\nProcessing law: {law.upper()}")
    
    for qa in law_info['questions_answers']:
        question = qa['question']
        reference_answer = qa['answer']
        
        print(f"Querying for question: {question}")

        query_embedding = embed_query(question, model_name)

        results = query_pinecone_db(query_embedding, index, namespace=law, top_k=1)

        if results and 'matches' in results and results['matches']:
            retrieved_chunk_id = results['matches'][0]['id']
            
            if law in chunks_dict:
                retrieved_chunk_index = int(retrieved_chunk_id.split('_')[-1])
                retrieved_chunk = chunks_dict[law][retrieved_chunk_index]

                print(f"Retrieved chunk {retrieved_chunk_index} from {law.upper()}:")
                print(retrieved_chunk)

                summary = summarize_text_huggingface_with_retry(retrieved_chunk)
                print(summary)
                qa['summary'] = summary
                if summary:
                    answer_embedding = cosine_model.encode(reference_answer, convert_to_tensor=False)
                    summary_embedding = cosine_model.encode(summary, convert_to_tensor=False)
                    qa['answer_embedding'] = answer_embedding
                    qa['summary_embedding'] = summary_embedding
                    
                    cosine_similarity = calculate_cosine_similarity(answer_embedding, summary_embedding)
                    qa['cosine_similarity'] = cosine_similarity
                    
                    semantic_similarity = util.pytorch_cos_sim(
                        semantic_model.encode(reference_answer, convert_to_tensor=True),
                        semantic_model.encode(summary, convert_to_tensor=True)
                    ).item()
                    qa['semantic_similarity'] = semantic_similarity
                    
                    print(f"Question: {question}")
                    print(f"Summary: {summary}")
                    print(f"Cosine Similarity: {cosine_similarity:.4f}")
                    print(f"Semantic Similarity: {semantic_similarity:.4f}")
                else:
                    print(f"No summary generated for question: {question}")

            else:
                print(f"Error: '{law}' not found in chunks_dict.")
        else:
            print(f"No results found for {law.upper()} for the question: {question}")

averages = {}
for law, info in laws_info.items():
    total_cosine = 0
    total_semantic = 0
    count = 0

    for qa in info['questions_answers']:
        if 'cosine_similarity' in qa and 'semantic_similarity' in qa:
            total_cosine += qa['cosine_similarity']
            total_semantic += qa['semantic_similarity']
            count += 1
    
    if count > 0:
        avg_cosine = total_cosine / count
        avg_semantic = total_semantic / count
        averages[law] = {
            'average_cosine_similarity': avg_cosine,
            'average_semantic_similarity': avg_semantic
        }
        print(f"\nAverage Cosine Similarity for {law.upper()}: {avg_cosine:.4f}")
        print(f"Average Semantic Similarity for {law.upper()}: {avg_semantic:.4f}")
    else:
        print(f"\nNo valid similarity scores for {law.upper()}.")

In [ ]:
import time

def measure_and_average_query_latency(laws_info, model, tokenizer, top_k=1):
    latency_results = {law: [] for law in laws_info.keys()}  
    all_latencies = []

    for law, info in laws_info.items():
        print(f"\nMeasuring query latency for {law.upper()} namespace:")

        if not info.get('questions_answers', []):
            print(f"No questions found for law: {law}")
            continue

        for qa in info['questions_answers']:
            query = qa.get('question', None)
            if not query:
                print(f"Skipping missing query for law: {law}")
                continue

            try:
                query_embedding = get_embedding(query, model, tokenizer)
                print(f"Generated embedding for query: {query}")
            except Exception as e:
                print(f"Error generating embedding for query: {query}. Error: {e}")
                continue

            start_time = time.time()

            try:
                results = index.query(
                    vector=query_embedding.tolist(), 
                    top_k=top_k,
                    namespace=law
                )
            except Exception as e:
                print(f"Error querying Pinecone for {query}: {e}")
                continue

            end_time = time.time()

            latency = end_time - start_time
            latency_results[law].append(latency)
            all_latencies.append(latency)

            print(f"Query: {query}")
            print(f"Latency: {latency:.4f} seconds")
            print("----\n")
    
    for law in latency_results:
        print(f"Latencies for {law}: {latency_results[law]}")
        if latency_results[law]: 
            avg_latency = sum(latency_results[law]) / len(latency_results[law])
            print(f"{law.upper()} Average Query Latency: {avg_latency:.4f} seconds")
        else:
            print(f"{law.upper()} has no recorded latencies.")

    if all_latencies:
        overall_avg_latency = sum(all_latencies) / len(all_latencies)
        print(f"\nOverall Average Query Latency: {overall_avg_latency:.4f} seconds")
    else:
        print("No latencies recorded across all laws.")

def get_embedding(text, model, tokenizer, max_length=512):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_length
    )
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

from transformers import AutoTokenizer, AutoModel

embedding_model = AutoModel.from_pretrained("microsoft/deberta-v3-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

measure_and_average_query_latency(laws_info, embedding_model, tokenizer, top_k=5)

# Load the other 80 questions, to retrieve the most relevant chunks, retrieve, making sums and compare them with the answers

In [ ]:
import pandas as pd

def inspect_csv(file_path, delimiter=","):
    try:
        df = pd.read_csv(file_path, encoding='utf-8', delimiter=delimiter, quotechar='"', skip_blank_lines=True)
        
        print("Column Names:", df.columns)
        
        print("\nFirst 5 Rows:\n", df.head())
        
        print("\nData Types:\n", df.dtypes)
        
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")

gdpr_csv_path = '/kaggle/input/greek-laws-and-qa/questions_answers_20gdpr_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv'
inspect_csv(gdpr_csv_path, delimiter=",") 

In [ ]:
import pandas as pd

def process_csv(file_path, output_path):
    try:
        print(f"\nProcessing file: {file_path}")

        df = pd.read_csv(file_path, encoding='utf-8', header=None) 

        print("First 5 Rows of Input:\n", df.head())

        questions = []
        answers = []
        current_question = None

        for _, row in df.iterrows():
            text = str(row[0]).strip()  

            if text.startswith("Ερώτηση:"):
                current_question = text.replace("Ερώτηση:", "").strip()
            elif text.startswith("Απάντηση:") and current_question:
                answer = text.replace("Απάντηση:", "").strip()
                questions.append(current_question)
                answers.append(answer)
                current_question = None  
            else:
                continue  

        qa_df = pd.DataFrame({'Ερώτηση': questions, 'Απάντηση': answers})

        print(f"Extracted DataFrame:\n{qa_df.head()}")

        qa_df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Processed file saved to: {output_path}")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")


file_paths = {
    'gdpr': ('/kaggle/input/greek-laws-and-qa/questions_answers_20gdpr_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv', '/kaggle/working/processed_gdpr_Pinecone_greek.csv'),
    'ai_act': ('/kaggle/input/greek-laws-and-qa/questions_answers_ai_test_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv', '/kaggle/working/processed_ai_act_Pinecone_greek.csv'),
    'dma': ('/kaggle/input/greek-laws-and-qa/questions_answers_digital_marketing_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv', '/kaggle/working/processed_dma_Pinecone_greek.csv'),
    'dsa': ('/kaggle/input/greek-laws-and-qa/questions_answers_digital_services_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv', '/kaggle/working/processed_dsa_Pinecone_greek.csv'),
}

for law, paths in file_paths.items():
    input_path, output_path = paths
    process_csv(input_path, output_path)

In [ ]:
import pandas as pd

def load_and_split_questions_answers(file_path):
    df = pd.read_csv(file_path, encoding='utf-8', header=None)
    
    questions = []
    answers = []
    
    current_question = None 
    
    for index, row in df.iterrows():
        text = row[0] 
        
        if pd.isna(text):
            continue
        
        text = str(text).lstrip().replace('\xa0', ' ') 

        if text.startswith("Ερώτηση:"):
            current_question = text.replace("Ερώτηση:", "").strip() 
        elif text.startswith("Απάντηση:") and current_question:
            answer = text.replace("Απάντηση:", "").strip()
            questions.append(current_question)
            answers.append(answer)
            current_question = None  
    
    qa_df = pd.DataFrame({'question': questions, 'answer': answers})
    
    return qa_df

csv_paths = {
    'gdpr': '/kaggle/input/greek-laws-and-qa/questions_answers_20gdpr_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv',
    'ai_act': '/kaggle/input/greek-laws-and-qa/questions_answers_ai_test_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv',
    'dma': '/kaggle/input/greek-laws-and-qa/questions_answers_digital_marketing_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv',
    'dsa': '/kaggle/input/greek-laws-and-qa/questions_answers_digital_services_20_CE_B5_CE_BB_CE_BB_CE_B7_CE_BD_CE_B9_CE_BA_CE_B1.csv'
}

qa_dataframes = {}

for law, path in csv_paths.items():
    qa_dataframes[law] = load_and_split_questions_answers(path)
    print(f"Processed {law.upper()} law:")
    print(qa_dataframes[law].head(), "\n")

In [ ]:
import pandas as pd
import pinecone
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch



device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Loading models...")
summarizer = pipeline("summarization", model="IMISLab/GreekT5-mt5-small-greeksum", device=0 if torch.cuda.is_available() else -1)
cosine_model = SentenceTransformer('nlpaueb/bert-base-greek-uncased-v1', device=device)
semantic_model = SentenceTransformer('paraphrase-mpnet-base-v2')', device=device)

torch.cuda.empty_cache()

def store_embeddings_in_pinecone(chunks, embeddings, namespace):
    vectors = [
        (f"{namespace}_id_{i}", embedding.tolist(), {"text": chunk})
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))
    ]
    index.upsert(vectors=vectors, namespace=namespace)

def embed_and_query(query_text, model_norm, namespace, top_k=1):
    try:
        query_embedding = model_norm.encode([query_text])[0]
        print(f"Query embedding created for: {query_text}")

        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            namespace=namespace
        )
        return results
    except Exception as e:
        print(f"Error querying Pinecone for namespace '{namespace}': {e}")
        return None


def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

def calculate_semantic_similarity(text1, text2):
    embedding1 = semantic_model.encode(text1)
    embedding2 = semantic_model.encode(text2)
    return cosine_similarity([embedding1], [embedding2])[0][0]

def summarize_and_compare_all_laws(qa_dataframes, model_norm, namespaces):
    similarities = {law: {'cosine': [], 'semantic': []} for law in namespaces.keys()}

    for law, df in qa_dataframes.items():
        print(f"\nProcessing summaries for {law.upper()} law:")
        question_col = 'Ερώτηση'
        answer_col = 'Risposta'

        for _, row in df.iterrows():
            question_text = row[question_col]
            answer_text = row[answer_col]
        
            print(f"Querying for: {question_text}")
            try:
                results = embed_and_query(question_text, model_norm, namespaces[law], top_k=1)
                if results and 'matches' in results and results['matches']:
                    retrieved_chunk_id = results['matches'][0]['id']
                if law in chunks_dict:
                    retrieved_chunk_index = int(retrieved_chunk_id.split('_')[-1])
                    retrieved_chunk = chunks_dict[law][retrieved_chunk_index]
    
                    print(f"Retrieved chunk {retrieved_chunk_index} from {law.upper()}:")
                    print(retrieved_chunk)
            
            except Exception as e:
                print(f"Error processing question '{question_text}': {e}")
                continue

        
            try:
                summary = summarizer(retrieved_chunk, max_length=350, min_length=100, do_sample=False)[0]['summary_text']
            except Exception as e:
                print(f"Failed to summarize chunk for question: {question_text}. Error: {e}")
                continue
        
            if summary:
                print(f"Original answer: {answer_text}")
                print(f"Retrieved chunk: {retrieved_chunk}")
                print(f"Generated summary: {summary}\n")
        
                try:
                    answer_embedding = model_norm.encode([answer_text])[0]
                    summary_embedding = model_norm.encode([summary])[0]
        
                    cosine_sim = calculate_cosine_similarity(answer_embedding, summary_embedding)
                    semantic_sim = calculate_semantic_similarity(answer_text, summary)
        
                    similarities[law]['cosine'].append(cosine_sim)
                    similarities[law]['semantic'].append(semantic_sim)
        
                    print(f"Cosine similarity: {cosine_sim:.4f}")
                    print(f"Semantic similarity: {semantic_sim:.4f}\n")
                except Exception as e:
                    print(f"Error calculating similarities for question: {question_text}. Error: {e}")
            else:
                print(f"Failed to summarize for: {question_text}")

    print("Calculated Averages:")
    for law in similarities:
        cosine_scores = similarities[law]['cosine']
        semantic_scores = similarities[law]['semantic']
        if cosine_scores:
            avg_cosine = sum(cosine_scores) / len(cosine_scores)
            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
        if semantic_scores:
            avg_semantic = sum(semantic_scores) / len(semantic_scores)
            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
    return similarities

qa_dataframes = {
    'gdpr': pd.read_csv('/kaggle/working/processed_gdpr_Pinecone_greek.csv'),
    'ai_act': pd.read_csv('/kaggle/working/processed_ai_act_Pinecone_greek.csv'),
    'dma': pd.read_csv('/kaggle/working/processed_dma_Pinecone_greek.csv'),
    'dsa': pd.read_csv('/kaggle/working/processed_dsa_Pinecone_greek.csv'),
}

for law, df in qa_dataframes.items():
    chunks = df['Απάντηση'].tolist()
    embeddings = cosine_model.encode(chunks)
    store_embeddings_in_pinecone(chunks, embeddings, namespace=law)

namespaces = {law: law for law in qa_dataframes.keys()}
summarize_and_compare_all_laws(qa_dataframes, cosine_model, namespaces)